<a href="https://colab.research.google.com/github/SujithSaiSripadam/Deep_Learning/blob/main/Copy_of_Cambridge_Code_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing necessary Libraries

In [1]:
pip install dgl torch-geometric optuna > /dev/null 2>&1

In [3]:
# importing libraries
import os
import pdb
import copy
import shutil
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold,LeaveOneOut

import scipy.io
from scipy.signal import resample
from scipy.fftpack import fft
from scipy.io import loadmat
from collections import OrderedDict



import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch, Data
from torch_geometric.utils import sparse, to_torch_coo_tensor
import torch_geometric.transforms as T
from torch_geometric.nn import GATv2Conv,GATConv, global_mean_pool, GCNConv
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

import optuna
from optuna.samplers import TPESampler
import dgl
import dgl.function as fn
import networkx as nx

ImportError: cannot import name 'DILL_AVAILABLE' from 'torch.utils.data.datapipes.utils.common' (/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/utils/common.py)

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set a specific seed value (replace 42 with your desired seed)
seed_value = 42
set_seed(seed_value)

# Data Pre-processing  1 ( When you dont have preprocess data)


## For Video Data


In [ ]:
import shutil
import os

# Source directory
source_dir = "/content/drive/MyDrive/"

# Destination directory
destination_dir = "/content/drive/MyDrive/Cambridge_Data"
#shutil.copytree(source_dir, destination_dir)

In [ ]:
# # Demo of Cambridge Original Data
# x=pd.read_csv("/content/drive/MyDrive/Cambridge_Data/Ayush/Data/s03_Task2.csv")
# print(x.head())
# print(x.shape)
# print(x.columns)

In [ ]:
#Data -> OpenFace_data
directory_path="/content/drive/MyDrive/Cambridge_Data/OpenFace_data"
Task_4=[]
for file in os.listdir(directory_path):
   if "Task4" in file:
      Task_4.append(file)
print(Task_4)
Task_4.sort()
print(Task_4)

['s03_Task4.csv', 's04_Task4.csv', 's05_Task4.csv', 's06_Task4.csv', 's07_Task4.csv', 's08_Task4.csv', 's10_Task4.csv', 's11_Task4.csv', 's13_Task4.csv', 's14_Task4.csv', 's15_Task4.csv', 's16_Task4.csv', 's24_Task4.csv', 's22_Task4.csv', 's21_Task4.csv', 's20_Task4.csv', 's17_Task4.csv', 's19_Task4.csv', 's26_Task4.csv', 's25_Task4.csv', 's41_Task4.csv', 's37_Task4.csv', 's38_Task4.csv', 's28_Task4.csv', 's36_Task4.csv', 's30_Task4.csv', 's39_Task4.csv', 's40_Task4.csv', 's34_Task4.csv', 's35_Task4.csv', 's43_Task4.csv', 's31_Task4.csv', 's32_Task4.csv', 's29_Task4.csv', 's33_Task4.csv', 's42_Task4.csv', 's010_Task4_P3.csv', 's03_Task4_P1.csv', 's08_Task4_P1.csv', 's11_Task4_P1.csv', 's07_Task4_P1.csv', 's04_Task4_P1.csv', 's05_Task4_P1.csv', 's06_Task4_P1.csv', 's14_Task4_P1.csv', 's20_Task4_P1.csv', 's21_Task4_P1.csv', 's16_Task4_P1.csv', 's15_Task4_P1.csv', 's17_Task4_P1.csv', 's19_Task4_P1.csv', 's24_Task4_P1.csv', 's28_Task4_P1.csv', 's25_Task4_P1.csv', 's22_Task4_P1.csv', 's29_T

In [ ]:
#  Modify csv to take only essential columns from it
def modify_df(df):
      df_new =df[[" gaze_0_x"," gaze_0_y"," gaze_0_z"," gaze_1_x"," gaze_1_y"," gaze_1_z"," gaze_angle_x"," gaze_angle_y"," pose_Tx"," pose_Ty"," pose_Tz"," pose_Rx"," pose_Ry"," pose_Rz"," AU01_r"," AU02_r"," AU04_r"," AU05_r"," AU06_r"," AU07_r"," AU09_r"," AU10_r"," AU12_r"," AU14_r"," AU15_r"," AU17_r"," AU20_r"," AU23_r"," AU25_r"," AU26_r"," AU45_r"]]
      return df_new

In [ ]:
directory_path="/content/drive/MyDrive/Cambridge_Data/OpenFace_data"
Task_2=[]
for file in os.listdir(directory_path):
   if "Task2" in file:
      Task_2.append(file)
print(Task_2)
Task_2.sort()
print(Task_2)
# Split TASK 2 into 2 separate CSV representing Happy and Sad
timestamp_Task2_h=[63, 60, 70, 71, 102, 103, 110, 74, 86, 74, 70, 83, 75, 78, 91, 71, 73, 84, 78, 70, 70, 84, 72, 72, 71, 71, 74, 72, 70, 73, 69, 77, 98, 72, 94, 82]
timestamp_Task2_s=[63, 60, 70, 71, 107, 112, 126, 74, 86, 74, 70, 83, 75, 78, 91, 71, 73, 84, 78, 70, 70, 84, 72, 72, 71, 71, 74, 72, 72, 73, 69, 77, 98, 72, 94, 82]
# subject_id=[S03, S04, S05, S06, S07, S08, S10, S11, S13, S14, S15, S16, S17, S19, S20, S21, S22, S24, S25, S26, S28, S29, S30, S31, S32, S33, S34, S35, S36, S37, S38, S39, S40, S41, S42, S43]
subject_id=['S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17', 'S19', 'S20', 'S21', 'S22', 'S24', 'S25', 'S26', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S42', 'S43']
i=0
directory_path="/content/drive/MyDrive/Cambridge_Data/OpenFace_data"
for file in Task_2:
      df = pd.read_csv(directory_path+'/'+file)
      mask_1 = df[' timestamp'] <= timestamp_Task2_h[i]
      df_1 = df[mask_1]
      df_1=modify_df(df_1)
      df_1.to_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_2_h/'+f"{subject_id[i]}.csv")
      mask_2 = df[' timestamp'] >= timestamp_Task2_s[i]
      df_2 = df[mask_2]
      df_2=modify_df(df_2)
      df_2.to_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_2_s/'+f"{subject_id[i]}.csv")
      i+=1
      print(i)

['s03_Task2.csv', 's05_Task2.csv', 's06_Task2.csv', 's07_Task2.csv', 's08_Task2.csv', 's10_Task2.csv', 's11_Task2.csv', 's13_Task2.csv', 's14_Task2.csv', 's15_Task2.csv', 's16_Task2.csv', 's17_Task2.csv', 's24_Task2.csv', 's26_Task2.csv', 's19_Task2.csv', 's25_Task2.csv', 's21_Task2.csv', 's22_Task2.csv', 's20_Task2.csv', 's35_Task2.csv', 's42_Task2.csv', 's40_Task2.csv', 's29_Task2.csv', 's30_Task2.csv', 's33_Task2.csv', 's34_Task2.csv', 's36_Task2.csv', 's38_Task2.csv', 's41_Task2.csv', 's39_Task2.csv', 's28_Task2.csv', 's43_Task2.csv', 's04_Task2.csv', 's31_Task2.csv', 's32_Task2.csv', 's37_Task2.csv']
['s03_Task2.csv', 's04_Task2.csv', 's05_Task2.csv', 's06_Task2.csv', 's07_Task2.csv', 's08_Task2.csv', 's10_Task2.csv', 's11_Task2.csv', 's13_Task2.csv', 's14_Task2.csv', 's15_Task2.csv', 's16_Task2.csv', 's17_Task2.csv', 's19_Task2.csv', 's20_Task2.csv', 's21_Task2.csv', 's22_Task2.csv', 's24_Task2.csv', 's25_Task2.csv', 's26_Task2.csv', 's28_Task2.csv', 's29_Task2.csv', 's30_Task2.c

In [ ]:
# Modify all csv files of Task 3
directory_path="/content/drive/MyDrive/Cambridge_Data/OpenFace_data"
subject_id=['S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17', 'S19', 'S20', 'S21', 'S22', 'S24', 'S25', 'S26', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S42', 'S43']
i=0
Task_3=[]
for file in os.listdir(directory_path):
   if "Task3" in file:
      Task_3.append(file)
print(Task_3)
Task_3.sort()
print(Task_3)

['s03_Task3.csv', 's04_Task3.csv', 's05_Task3.csv', 's07_Task3.csv', 's08_Task3.csv', 's11_Task3.csv', 's14_Task3.csv', 's15_Task3.csv', 's16_Task3.csv', 's21_Task3.csv', 's25_Task3.csv', 's17_Task3.csv', 's26_Task3.csv', 's24_Task3.csv', 's20_Task3.csv', 's22_Task3.csv', 's40_Task3.csv', 's28_Task3.csv', 's36_Task3.csv', 's33_Task3.csv', 's39_Task3.csv', 's41_Task3.csv', 's38_Task3.csv', 's35_Task3.csv', 's30_Task3.csv', 's29_Task3.csv', 's43_Task3.csv', 's32_Task3.csv', 's19_Task3.csv', 's34_Task3.csv', 's42_Task3.csv', 's37_Task3.csv', 's06_Task3.csv', 's10_Task3.csv', 's13_Task3.csv', 's31_Task3.csv']
['s03_Task3.csv', 's04_Task3.csv', 's05_Task3.csv', 's06_Task3.csv', 's07_Task3.csv', 's08_Task3.csv', 's10_Task3.csv', 's11_Task3.csv', 's13_Task3.csv', 's14_Task3.csv', 's15_Task3.csv', 's16_Task3.csv', 's17_Task3.csv', 's19_Task3.csv', 's20_Task3.csv', 's21_Task3.csv', 's22_Task3.csv', 's24_Task3.csv', 's25_Task3.csv', 's26_Task3.csv', 's28_Task3.csv', 's29_Task3.csv', 's30_Task3.c

In [ ]:
# file=file[30:]
directory_path="/content/drive/MyDrive/Cambridge_Data/OpenFace_data"
for file in Task_3:
      df_1 = pd.read_csv(directory_path+'/'+file)
      df_1=modify_df(df_1)
      df_1.to_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task3_final/'+f"{subject_id[i]}.csv")
      i+=1
      print(i," subject id:", subject_id[i-1])

NameError: name 'Task_3' is not defined

In [ ]:
dir_path="/content/drive/MyDrive/Cambridge_Data/Modify_Audio_Data/Task5p_A"
len(os.listdir(dir_path))

In [ ]:
# Modify all csv files of Task 5
directory_path="/content/drive/MyDrive/Cambridge_Data/Data"
subject_id=['S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17', 'S19', 'S20', 'S21', 'S22', 'S24', 'S25', 'S26', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S42', 'S43']
i=0
Task_5=[]
for file in os.listdir(directory_path):
   if "Task5" in file:
      Task_5.append(file)
print(Task_5)
Task_5.sort()
print(Task_5)
# file=file[0:]
for file in Task_5:
      df_1 = pd.read_csv(directory_path+'/'+file)
      df_1=modify_df(df_1)
      df_1.to_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_5/'+f"{subject_id[i]}.csv")
      i+=1
      print(i)

In [ ]:
#  Checking directory length
len(os.listdir('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_4_P3'))

In [ ]:
#  For modification of files which are not converted in loop due to some problem in file
file='/content/drive/MyDrive/Cambridge_Data/OpenFace_data/s19_Task3.csv'
directory_path="/content/drive/MyDrive/Cambridge_Data/OpenFace_data"
subject_id=19
df_1 = pd.read_csv(file)
df_1=modify_df(df_1)
df_1.to_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_3/S'+f"{subject_id}.csv")

KeyError: "None of [Index([' gaze_0_x', ' gaze_0_y', ' gaze_0_z', ' gaze_1_x', ' gaze_1_y',\n       ' gaze_1_z', ' gaze_angle_x', ' gaze_angle_y', ' pose_Tx', ' pose_Ty',\n       ' pose_Tz', ' pose_Rx', ' pose_Ry', ' pose_Rz', ' AU01_r', ' AU02_r',\n       ' AU04_r', ' AU05_r', ' AU06_r', ' AU07_r', ' AU09_r', ' AU10_r',\n       ' AU12_r', ' AU14_r', ' AU15_r', ' AU17_r', ' AU20_r', ' AU23_r',\n       ' AU25_r', ' AU26_r', ' AU45_r'],\n      dtype='object')] are in the [columns]"

In [ ]:
directory_path="/content/drive/MyDrive/Cambridge_Data/Data"
Task_4=[]
for file in os.listdir(directory_path):
   if "Task4" in file:
      Task_4.append(file)
print(Task_4)
Task_4.sort()
print(Task_4)
# Split TASK 4 into 3 separate CSV representing Picture 1,2 and 3
timestamp_Task4_1=[103, 107, 132, 139, 142, 158, 167, 131, 151, 140, 157, 129, 146, 131, 152, 129, 138, 136, 153, 130, 140, 122, 136, 145, 136, 99, 131, 152, 146, 137, 130, 131, 133, 131, 144, 132] #Between start to picture 1
timestamp_Task4_2=[172, 164, 203, 239, 215, 271, 252, 201, 243, 211, 237, 204, 236, 202, 235, 201, 210, 212, 232, 203, 210, 208, 209, 218, 218, 176, 206, 228, 222, 212, 202, 201, 207, 208, 224, 216] # Start of picture 3 to end
subject_id=['S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17', 'S19', 'S20', 'S21', 'S22', 'S24', 'S25', 'S26', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S42', 'S43']
i=0
# file=file[21:22]
directory_path="/content/drive/MyDrive/Cambridge_Data/Data"
for file in Task_4:
      df = pd.read_csv(directory_path+'/'+file)
      mask_1 = df[' timestamp']  <= timestamp_Task4_1[i]
      df_1 = df[mask_1]
      df_1=modify_df(df_1)
      df_1.to_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task4_P1_final/'+f"{subject_id[i]}.csv")
      mask_2 = ((df[' timestamp']  > timestamp_Task4_1[i]) & (df[' timestamp']  <= timestamp_Task4_2[i]))
      df_2 = df[mask_2]
      df_2=modify_df(df_2)
      df_2.to_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task4_P2_final/'+f"{subject_id[i]}.csv")
      mask_3 = df[' timestamp']  > timestamp_Task4_2[i]
      df_3 = df[mask_3]
      df_3=modify_df(df_3)
      df_3.to_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task4_P3_final/'+f"{subject_id[i]}.csv")
      i+=1
      print(i," Subject id:", subject_id[i-1])

In [ ]:
#  For modification of files which are not converted in loop due to some problem in file
file='/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_3/S03.csv'
subject_id=10
df_1 = pd.read_csv(file)
# df_1=df_1.drop('Unnamed: 0', axis=1)
df_1.to_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_3/S'+f"{subject_id}.csv", index=False)

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_3/S03.csv', index_col=[0])
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_3/S03.csv'

## For audio Data

In [ ]:
file_list=[]
def list_files_in_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_list.append(file_path)

# Replace 'your_directory_path' with the path of the directory you want to explore
directory_path = '/content/drive/MyDrive/Cambridge_Data/Audio_data'

list_files_in_directory(directory_path)
file_list=sorted(file_list)
file_list

['/content/drive/MyDrive/Cambridge_Data/Audio_data/.DS_Store',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s10/v1Task2h.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s10/v1Task2s.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s10/v1Task3p.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s10/v1Task4_P1.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s10/v1Task4_P2.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s10/v1Task4_P3.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s10/v1Task5p.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s11/v1Task2h.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s11/v1Task2s.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s11/v1Task3p.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s11/v1Task4_P1.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s11/v1Task4_P2.csv',
 '/content/drive/MyDrive/Cambridge_Data/Audio_data/s11/v1Task4_P3.csv',
 '/content/driv

In [ ]:
# Task 2
Task2h_A=[]
Task2s_A=[]
Task3_A=[]
Task4_P1_A=[]
Task4_P2_A=[]
Task4_P3_A=[]
Task5_A=[]
for file in file_list:
   if 'Task2h' in file:
    Task2h_A.append(file)
   elif 'Task2s' in file:
    Task2s_A.append(file)
   elif 'Task3p' in file:
    Task3_A.append(file)
   elif 'Task4_P1' in file:
    Task4_P1_A.append(file)
   elif 'Task4_P2' in file:
    Task4_P2_A.append(file)
   elif 'Task4_P3' in file:
    Task4_P3_A.append(file)
   elif 'Task5p' in file:
    Task5_A.append(file)

# Task2h_A.sort(); Task2s_A.sort(); Task3_A.sort(); Task4_P1_A.sort(); Task4_P2_A.sort(); Task4_P3_A.sort() ; Task5_A.sort()
print(len(Task2h_A), len(Task2s_A), len(Task3_A),len(Task4_P1_A), len(Task4_P2_A), len(Task4_P3_A),len(Task5_A))

36 36 36 36 36 36 36


In [ ]:
import shutil
subject_id=['S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17', 'S19', 'S20', 'S21', 'S22', 'S24', 'S25', 'S26', 'S28', 'S29', 'S30', 'S31', 'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41', 'S42', 'S43']
def move_files(file_paths, destination_directory):
    i=0
    for file_path in file_paths:
        dest =os.path.join(destination_directory+'/'+ subject_id[i])
        shutil.move(file_path, dest)
        i+=1
destination_directory = '/content/drive/MyDrive/Cambridge_Data/Modify_Audio_Data/Task5p_A'
move_files(Task5_A, destination_directory)
len(os.listdir(destination_directory))

36

# Data Preprocessing -2 (Used when you have preprocess data)

In [ ]:
# Preprocess function, We can chnage this as per our requirement
def preprocess(raw_data):
    num, length = raw_data.shape
    out_data = np.zeros((num, length))
    for i in range(num):
        out_data[i, :] = raw_data.iloc[i] - np.median(raw_data.iloc[i])
    return out_data

In [ ]:
def getVideoFeature(feaVec):
    feaNum = feaVec.shape[0]
    videoFea = np.zeros((1, feaNum*20-6*6))
    # print(videoFea.shape)
    for i in range(feaNum):
        videoFea[0, (i-1)*12 + 1] = np.mean(feaVec[i,:])
        videoFea[0, (i-1)*12 + 2] = np.std(feaVec[i,:])
        videoFea[0, (i-1)*12 + 3] = np.max(feaVec[i,:])
        videoFea[0, (i-1)*12 + 4] = np.min(feaVec[i,:])
        videoFea[0, (i-1)*12 + 5] = np.mean(np.diff(feaVec[i,:]))
        videoFea[0, (i-1)*12 + 6] = np.std(np.diff(feaVec[i,:]))
        videoFea[0, (i-1)*12 + 7] = np.max(np.diff(feaVec[i,:]))
        videoFea[0, (i-1)*12 + 8] = np.min(np.diff(feaVec[i,:]))
        a = feaVec[i,0:-2]
        b = feaVec[i,2:]
        videoFea[0, (i-1)*12 + 9] = np.mean(a-b)
        videoFea[0, (i-1)*12 + 10] = np.std(a-b)
        videoFea[0, (i-1)*12 + 11] = np.max(a-b)
        videoFea[0, (i-1)*12 + 12] = np.min(a-b)
    return videoFea

In [ ]:
def flat_data(data, t_length, N):
    flat_data = np.zeros(t_length)
    row_num = data.shape[0]
    for i in range(row_num):
        flat_data[i * N:(i + 1) * N] = data[i, :]
    return flat_data

In [ ]:
def cut_videos(raw_data, fre_resolution):
    num_frames = raw_data.shape[1]
    # raw_data=raw_data.astype(int)
    # print(raw_data.shape)
    raw_num_keep_frame = np.floor(num_frames /fre_resolution)
    num_keep_frame = raw_num_keep_frame * fre_resolution
    num_delete = num_frames - num_keep_frame

    # num_delete_start = np.ceil(num_delete / 2)
    # num_delete_end = np.floor(num_delete / 2)
    num_delete_start = num_delete // 2
    num_delete_end = num_delete - num_delete_start
    # print(raw_data.shape)

    # Remove the specified number of frames from the beginning and end
    raw_data = np.delete(raw_data, np.r_[:num_delete_start, num_frames - num_delete_end:].astype(int), axis=1)
    # raw_data = np.delete(raw_data, np.concatenate((np.arange(0, num_delete_start), np.arange(num_frames - num_delete_end, num_frames))).astype(int), axis=1)

    return raw_data, num_keep_frame, raw_num_keep_frame

In [ ]:
# Plotting heat map
def plot_heatmap(amp_map):
  print(amp_map.shape)
  print(amp_map)
  dataplot=sns.heatmap(amp_map, cmap='Reds')
  plt.show()

# Conversion of Time-series to spectral data

##First Frequency Alignment (Not using currently)

In [ ]:
'''

# first frequency alignment method (select) - Zero- Padding
def fourier_transform_select(all_data, N, num_multiple, fre_resolution):
    channel_num, length = all_data.shape
    amp_map = np.zeros((channel_num, length))
    phase_map = np.zeros((channel_num, length))
    common_temp_amp = np.zeros((channel_num, fre_resolution))
    common_temp_pha = np.zeros((channel_num, fre_resolution))

    for i in range(channel_num):
        temp_contain = np.fft.fft(all_data[i,:])
        amp_map[i,:] = np.abs(temp_contain/length)
        phase_map[i,:] = np.angle(temp_contain)

    for j in range(fre_resolution):
        common_temp_amp[:,j] = amp_map[:,1+(j-1)*num_multiple]
        common_temp_pha[:,j] = phase_map[:,1+(j-1)*num_multiple]

    amp_map_return = np.column_stack((amp_map[:,int(length//2)+1], common_temp_amp[:,1:N]))
    phase_map_return = np.column_stack((phase_map[:,int(length//2)+1], common_temp_pha[:,1:N]))

    return amp_map_return, phase_map_return

# demo for extracting spectral map and features using the first frequency alignment method (select)
Primitive_num = 18
N = 80
fre_resolution = 256

# pre_processing
t_length = N * Primitive_num
raw_data_1=pd.read_csv("/content/Cambridge_Data_Modify/s05_Task2 - Output.csv")
print(raw_data_1.shape)
raw_data, num_keep_frame, num_multiple = cut_videos(raw_data_1, fre_resolution)
processed_data_1 = preprocess(raw_data_1)
processed_data_1 = processed_data_1.T
print(processed_data_1.shape)
# feature extraction
sta_fea = getVideoFeature(processed_data_1)
amp_map_1, phase_map_1 = fourier_transform_select(processed_data_1, N, int(num_multiple), fre_resolution)
print(amp_map_1.shape, phase_map_1.shape)
# amp_flat_data = flat_data(amp_map_1, t_length, N)
# phase_flat_data = flat_data(phase_map_1, t_length, N)

'''

## Second Frequency Alignment ( Using Currently)

In [ ]:
# second frequency alignment method (resample)- extracts fixed-size spectral signals from variable length time-series data by choosing k common frequencies from
# the spectral signals obtained from each video. In this case, the values of k chosen frequencies are obtained from the original signal rather than an extended signal
def fourier_transform_resample(all_data, N, num_fre):
    channel_num, length = all_data.shape
    amp_map = np.zeros((channel_num, num_fre))
    phase_map = np.zeros((channel_num, num_fre))
    # print(all_data.shape, amp_map.shape, phase_map.shape)

    for i in range(channel_num):
        # print(np.fft.fft(all_data[i,:].shape))
        temp_contain = np.fft.fft(all_data[i,:])
        # print(temp_contain.ndim)
        # print(temp_contain.shape)

        if length % 2 == 0:
            temp_contain = temp_contain[:length//2+1]
        else:
            temp_contain = temp_contain[:(length+1)//2]

        temp_resample_data = resample(temp_contain, num_fre)
        amp_map[i,:] = np.abs(temp_resample_data) / length
        phase_map[i,:] = np.angle(temp_resample_data)

    amp_map_return = amp_map[:, :N]
    phase_map_return = phase_map[:, :N]

    return amp_map_return, phase_map_return

In [ ]:
# Applying 2nd frequency alignment method to the csv file containing temporal data of 1 of the sbject for task1 and then coverting this data to spectral
# representaion in form of spectral heatmap

# pre_processing
def generate_spectral_heatmap(dir_path,name, Primitive_num, K = 60, fre_resolution = 128):
  t_length = K * Primitive_num
  # raw_data = scipy.io.loadmat(file_name)['example_data']
  # raw_data=pd.read_csv("/content/Cambridge_Data_Modify/s05_Task2 - Output.csv")
  filename=dir_path+'/'+ name
  raw_data=pd.read_csv(filename, index_col=[0])
  # print(raw_data.shape)
  # print(raw_data.iloc[2]-np.median(raw_data.iloc[2]))
  processed_data = preprocess(raw_data)
  processed_data = processed_data.T

  # feature extraction
  sta_fea = getVideoFeature(processed_data)
  amp_map, phase_map = fourier_transform_resample(processed_data,K, fre_resolution)
  # plot_heatmap(amp_map)
  # plot_heatmap(phase_map)
  # amp_flat_data = amp_map.flatten()[:t_length]
  # phase_flat_data = phase_map.flatten()[:t_length]
  spectral_data= np.concatenate((amp_map, phase_map), axis=1)
  # print(spectral_data.shape)
  df_spectral_data=pd.DataFrame(spectral_data)
  # print(df_spectral_data)
  return df_spectral_data

In [ ]:
def graph_visualization(G):
  pos = nx.spring_layout(G)
  plt.title("DGL Graph Visualization")
  nx.draw(G,pos, with_labels=True, node_color = 'green',node_size = 1500, font_size=10)

# Necessary modules for Multi-dimenisonal Graph Geeneration

##MEFG Module Layers(GRATIS)

In [ ]:
1# Classes of Layers used in MEFG module of GRATIS
"""
    ResGatedGCN: Residual Gated Graph ConvNets
    An Experimental Study of Neural Networks for Variable Graphs (Xavier Bresson and Thomas Laurent, ICLR 2018)
    https://arxiv.org/pdf/1711.07553v2.pdf
"""

class GatedGCNLayer(nn.Module):
    """
        Param: []
    """
    # input_dim= 120, output_dim= 240
    def __init__(self, input_dim, output_dim, dropout, batch_norm, residual=False):
        super().__init__()
        self.in_channels = input_dim
        self.out_channels = output_dim
        self.dropout = dropout
        self.batch_norm = batch_norm
        self.residual = residual

        if input_dim != output_dim:
            self.residual = False
        #  self.residual = False
        self.A = nn.Linear(input_dim, output_dim, bias=True)
        self.B = nn.Linear(input_dim, output_dim, bias=True)
        self.D = nn.Linear(input_dim, output_dim, bias=True)
        self.E = nn.Linear(input_dim, output_dim, bias=True)
        self.bn_node_h = nn.BatchNorm1d(output_dim)
        self.bn_node_e = nn.BatchNorm1d(output_dim)
        self.C = nn.Linear(input_dim, output_dim, bias=True)

    def forward(self, g, h, e):

        h_in = h # for residual connection
        e_in = e # for residual connection
        # print("node matrix shape :", h.shape,"edge matrix shape:", e.shape)
        g.ndata['h']  = h
        # h=h.T
        g.ndata['Ah'] = self.A(h)
        g.ndata['Bh'] = self.B(h)
        g.ndata['Dh'] = self.D(h)
        g.ndata['Eh'] = self.E(h)
        g.edata['e']  = e
        # g.edata['Ce'] = self.C(e)

        g.apply_edges(fn.u_add_v('Dh', 'Eh', 'DEh'))
        # print("Size of DEh :", g.edata['DEh'].shape)
        # g.edata['e'] = g.edata['DEh'] + g.edata['Ce'] # Original code
        g.edata['e'] = g.edata['DEh'] + e # Changed code excluding g.edata['Ce']
        g.edata['sigma'] = torch.sigmoid(g.edata['e'])
        g.update_all(fn.u_mul_e('Bh', 'sigma', 'm'), fn.sum('m', 'sum_sigma_h'))
        g.update_all(fn.copy_e('sigma', 'm'), fn.sum('m', 'sum_sigma'))
        g.ndata['h'] = g.ndata['Ah'] + g.ndata['sum_sigma_h'] / (g.ndata['sum_sigma'] + 1e-6)
        #g.update_all(self.message_func,self.reduce_func)
        h = g.ndata['h'] # result of graph convolution
        e = g.edata['e'] # result of graph convolution
        # print("h shape:", h.shape,"e shape :", e.shape)

        if self.batch_norm:
            h = self.bn_node_h(h) # batch normalization
            e = self.bn_node_e(e) # batch normalization

        h = F.relu(h) # non-linear activation
        e = F.relu(e) # non-linear activation

        if self.residual:
            h = h_in + h # residual connection
            e = e_in + e # residual connection
        h = F.dropout(h, self.dropout, training=self.training)
        e = F.dropout(e, self.dropout, training=self.training)
        # print("h shape:", h.shape,"e shape :", e.shape)
        return h, e

    def __repr__(self):
        return '{}(in_channels={}, out_channels={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels)



In [ ]:
'''
Attention with bool_masked_pos argument.
'''
class CrossAttention(nn.Module):
    def __init__(
            self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0.,
            proj_drop=0., window_size=None, attn_head_dim=None):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        if attn_head_dim is not None:
            head_dim = attn_head_dim
        all_head_dim = head_dim * self.num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.q = nn.Linear(dim, all_head_dim, bias=False)
        self.k = nn.Linear(dim, all_head_dim, bias=False)
        self.v = nn.Linear(dim, all_head_dim, bias=False)

        if qkv_bias:
            self.q_bias = nn.Parameter(torch.zeros(all_head_dim))
            # self.k_bias = nn.Parameter(torch.zeros(all_head_dim))
            self.v_bias = nn.Parameter(torch.zeros(all_head_dim))
        else:
            self.q_bias = None
            self.k_bias = None
            self.v_bias = None


        self.window_size = None
        self.relative_position_bias_table = None
        self.relative_position_index = None

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(all_head_dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x, bool_masked_pos=None, k=None, v=None):
        B, N, C = x.shape

        if k is None:
            k = x
            v = x
            N_k = N
            N_v = N
        else:
            N_k = k.shape[1]
            N_v = v.shape[1]

        q_bias, k_bias, v_bias = None, None, None
        if self.q_bias is not None:
            q_bias = self.q_bias
            k_bias = torch.zeros_like(self.v_bias, requires_grad=False)
            v_bias = self.v_bias

        q = F.linear(input=x, weight=self.q.weight, bias=q_bias)                        # (B, N_q, dim)
        k = F.linear(input=k, weight=self.k.weight, bias=k_bias)                        # (B, N_k, dim)
        v = F.linear(input=v, weight=self.v.weight, bias=v_bias)

        q = q.reshape(B, N, 1, self.num_heads, -1).permute(2, 0, 3, 1, 4).squeeze(0)    # (B, num_heads, N_q, dim)
        k = k.reshape(B, N_k, 1, self.num_heads, -1).permute(2, 0, 3, 1, 4).squeeze(0)    # (B, num_heads, N_k, dim)
        v = v.reshape(B, N_v, 1, self.num_heads, -1).permute(2, 0, 3, 1, 4).squeeze(0)    # (B, num_heads, N_v, dim)


        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))      # (B, N_head, N, N)

        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, -1)
        x = self.proj(x)
        x = self.proj_drop(x)

        return x






"""
Linear Transformer proposed in "Transformers are RNNs: Fast Autoregressive Transformers with Linear Attention"
Modified from: https://github.com/idiap/fast-transformers/blob/master/fast_transformers/attention/linear_attention.py
"""

def elu_feature_map(x):
    return torch.nn.functional.elu(x) + 1


class LinearAttention(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.feature_map = elu_feature_map
        self.eps = eps

    def forward(self, queries, keys, values, q_mask=None, kv_mask=None):
        """ Multi-Head linear attention proposed in "Transformers are RNNs"
        Args:
            queries: [N, L, H, D]
            keys: [N, S, H, D]
            values: [N, S, H, D]
            q_mask: [N, L]
            kv_mask: [N, S]
        Returns:
            queried_values: (N, L, H, D)
        """
        Q = self.feature_map(queries)
        K = self.feature_map(keys)

        # set padded position to zero
        if q_mask is not None:
            Q = Q * q_mask[:, :, None, None]
        if kv_mask is not None:
            K = K * kv_mask[:, :, None, None]
            values = values * kv_mask[:, :, None, None]

        v_length = values.size(1)
        values = values / v_length  # prevent fp16 overflow
        KV = torch.einsum("nshd,nshv->nhdv", K, values)  # (S,D)' @ S,V
        Z = 1 / (torch.einsum("nlhd,nhd->nlh", Q, K.sum(dim=1)) + self.eps)
        queried_values = torch.einsum("nlhd,nhdv,nlh->nlhv", Q, KV, Z) * v_length

        return queried_values.contiguous()


class FullAttention(nn.Module):
    def __init__(self, use_dropout=False, attention_dropout=0.1):
        super().__init__()
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(attention_dropout)

    def forward(self, queries, keys, values, q_mask=None, kv_mask=None):
        """ Multi-head scaled dot-product attention, a.k.a full attention.
        Args:
            queries: [N, L, H, D]
            keys: [N, S, H, D]
            values: [N, S, H, D]
            q_mask: [N, L]
            kv_mask: [N, S]
        Returns:
            queried_values: (N, L, H, D)
        """

        # Compute the unnormalized attention and apply the masks
        QK = torch.einsum("nlhd,nshd->nlsh", queries, keys)
        if kv_mask is not None:
            QK.masked_fill_(~(q_mask[:, :, None, None] * kv_mask[:, None, :, None]), float('-inf'))

        # Compute the attention and the weighted average
        softmax_temp = 1. / queries.size(3)**.5  # sqrt(D)
        A = torch.softmax(softmax_temp * QK, dim=2)
        if self.use_dropout:
            A = self.dropout(A)

        queried_values = torch.einsum("nlsh,nshd->nlhd", A, values)

        return queried_values.contiguous()


class CrossTransformerEncoder(nn.Module):
    def __init__(self,
                 d_model,
                 nhead,
                 attention='linear',
                 drop = 0.1):
        super(CrossTransformerEncoder, self).__init__()

        self.dim = d_model // nhead
        self.nhead = nhead

        # multi-head attention
        self.q_proj = nn.Linear(d_model, d_model, bias=False)
        self.k_proj = nn.Linear(d_model, d_model, bias=False)
        self.v_proj = nn.Linear(d_model, d_model, bias=False)
        self.attention = LinearAttention() if attention == 'linear' else FullAttention()
        self.merge = nn.Linear(d_model, d_model, bias=False)

        # feed-forward network
        self.mlp = nn.Sequential(
            nn.Linear(d_model*2, d_model*2, bias=False),
            #nn.Dropout(drop),
            nn.ReLU(True),
            nn.Linear(d_model*2, d_model, bias=False),
            nn.Dropout(drop),
        )

        # norm and dropout
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x, source, x_mask=None, source_mask=None):
        """
        Args:
            x (torch.Tensor): [N, L, C]
            source (torch.Tensor): [N, S, C]
            x_mask (torch.Tensor): [N, L] (optional)
            source_mask (torch.Tensor): [N, S] (optional)
        """
        bs = x.size(0)
        # print("shape of x:", x.shape," shape of source:",source.shape)
        query, key, value = x, source, source
        # print(" shape of query:", query.shape," shape of key:", key.shape," shape of value:", value.shape)
        # multi-head attention
        query = self.q_proj(query).view(bs, -1, self.nhead, self.dim)  # [N, L, (H, D)]
        # print(" query shape: [N, L, (H, D)]", query.shape)
        key = self.k_proj(key).view(bs, -1, self.nhead, self.dim)  # [N, S, (H, D)]
        # print("shape of key should be [N, S, (H, D)] :", key.shape)
        value = self.v_proj(value).view(bs, -1, self.nhead, self.dim)
        message = self.attention(query, key, value, q_mask=x_mask, kv_mask=source_mask)  # [N, L, (H, D)]
        # print("message shape should be [N, L, (H, D)] :", message.shape)
        message = self.merge(message.view(bs, -1, self.nhead*self.dim))  # [N, L, C]
        # print("message shape should be [N, L, C] :", message.shape)
        message = self.norm1(message)

        # feed-forward network
        message = self.mlp(torch.cat([x, message], dim=2))
        message = self.norm2(message)

        return x + message

class CrossTransformer(nn.Module):

    def __init__(self, d_model, nhead=1, layer_nums=1, attention_type='linear'):
        super().__init__()

        encoder_layer = CrossTransformerEncoder(d_model, nhead, attention_type)
        self.VCR_layers = nn.ModuleList([encoder_layer for _ in range(layer_nums)])
        self.VVR_layers = nn.ModuleList([encoder_layer for _ in range(layer_nums)])

        self._reset_parameters()

    def _reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, qfea, kfea, mask0=None, mask1=None):
        """
        Args:
            qfea (torch.Tensor): [B, N, D]
            kfea (torch.Tensor): [B, D]
            mask0 (torch.Tensor): [B, N] (optional)
            mask1 (torch.Tensor): [B, N] (optional)
        """
        #assert self.d_model == qfea.size(2), "the feature number of src and transformer must be equal"

        B,N,D = qfea.shape
        kfea = kfea.unsqueeze(1).repeat(1, N, 1) #[B,N,D]
        # print(" shape of kfea should be [B,N,D] :", kfea.shape)

        mask1 = torch.ones([B,N]).to(qfea.device)
        for layer in self.VCR_layers:
            qfea = layer(qfea, kfea, mask0, mask1) #[B,N,D]
            # print(" shape of qfea should be [B,N,D] :", qfea.shape)
            #kfea = layer(kfea, qfea, mask1, mask0)

        qfea_end = qfea.repeat(1,1,N).view(B,-1,D) #[B,N*N,D]
        # print(" shape of qfea end should be [B,N*N,D] :", qfea_end.shape)
        qfea_start = qfea.repeat(1,N,1).view(B,-1,D) #[B,N*N,D]
        # print(" shape of qfea start should be [B,N*N,D] :", qfea_start.shape)
        #mask2 = mask0.repeat([1,N])
        for layer in self.VVR_layers:
            #qfea_start = layer(qfea_start, qfea_end, mask2, mask2)
            qfea_start = layer(qfea_start, qfea_end)#[B,N*N,D]
            # print(" shape of qfea start should be [B,N*N,D] :", qfea_start.shape)

        return qfea_start.view([B,N,N,D])

## TTP Module (GRATIS)

In [ ]:
# The TTP code
class TTP(nn.Module):
    def __init__(self, in_dim, hidden_dim, edge_thresh):
        super().__init__()
        self.proj_g1 = nn.Linear(in_dim,hidden_dim**2)
        self.bn_node_lr_g1 = nn.BatchNorm1d(hidden_dim**2)
        self.proj_g2 = nn.Linear(in_dim,hidden_dim)
        self.bn_node_lr_g2 = nn.BatchNorm1d(hidden_dim)
        self.hidden_dim = hidden_dim #lr_g
        self.proj_g = nn.Linear(hidden_dim, 1)
        self.edge_thresh = edge_thresh
    def forward(self, g, h, e):
        lr_gs = []
        gs = dgl.unbatch(g)
        for g in gs:
            N = g.number_of_nodes()
            h_single = g.ndata['feature'].to(h.device)
            h_proj1 = F.dropout(F.relu(self.bn_node_lr_g1(self.proj_g1(h_single))), 0.1, training=self.training).view(-1,self.hidden_dim)
            h_proj2 = F.dropout(F.relu(self.bn_node_lr_g2(self.proj_g2(h_single))), 0.1, training=self.training).permute(1,0)
            mm = torch.mm(h_proj1,h_proj2)
            mm = mm.view(N,self.hidden_dim,-1).permute(0,2,1) #[N, N, D]
            # print("Golbal Contextual Representation shape:", mm.shape)

            mm = self.proj_g(mm).squeeze(-1)
            # print("Xh matrix shape:", mm.shape)
            diag_mm = torch.diag(mm)
            diag_mm = torch.diag_embed(diag_mm)
            mm -= diag_mm
            #matrix = torch.sigmoid(mm)
            #matrix = F.softmax(mm, dim=0)
            matrix = F.softmax(mm, dim=0) * F.softmax(mm, dim=1)
            # print("Adjacency probability matrix",matrix.shape)
            #binarized = BinarizedF()
            #matrix = binarized.apply(matrix) #(0/1)
            lr_connetion = torch.where(matrix>self.edge_thresh)
            # print(lr_connetion[0], lr_connetion[1])
            g.add_edges(lr_connetion[0], lr_connetion[1])
            # print("Learned TTP Graph", g)
            lr_gs.append(g)
        g = dgl.batch(lr_gs).to(h.device)

        return g

## MEFG Module

In [ ]:
'''
M =num_edges
Din= input_dim = 120
hidden_num= hidden dim =64
Nmax= num_nodes = 31
D= node_features =120
B = Batch
'''
class MEFG(nn.Module):
    def __init__(self, in_dim,hidden_dim, max_node_num, global_layer_num = 1, dropout = 0.1):
        super().__init__()
        self.edge_proj = nn.Conv1d(in_channels=2,out_channels=1,kernel_size=3,padding=1)
        self.edge_proj2 = nn.Linear(in_dim,hidden_dim) #baseline4
        self.edge_proj3 = nn.Linear(in_dim,hidden_dim)
        self.edge_proj4 = nn.Linear(hidden_dim,hidden_dim)
        self.hidden_dim = hidden_dim #baseline4
        self.bn_node_lr_e = nn.BatchNorm1d(hidden_dim)

        self.max_node_num = max_node_num
        self.global_layers = nn.ModuleList([GatedGCNLayer(in_dim, hidden_dim, dropout,
                                    True, True) for _ in range(global_layer_num -1) ])
        self.global_layers.append(GatedGCNLayer(in_dim, hidden_dim, dropout, True, True))
        #self.global_layers = nn.ModuleList([ ResidualAttentionBlock( d_model = hidden_dim, n_head = 1)
        #                                    for _ in range(global_layer_num) ])
        self.CrossT = CrossTransformer(hidden_dim, nhead=1, layer_nums=1, attention_type='linear')

    def forward(self, g, h, e):

        g.apply_edges(lambda edges: {'src' : edges.src['feature']})
        src = g.edata['src'].unsqueeze(1) #[M,1,D]
        # print("src shape:", src.shape)
        g.apply_edges(lambda edges: {'dst' : edges.dst['feature']})
        dst = g.edata['dst'].unsqueeze(1) #[M,1,D]
        # print("dst shape:", dst.shape)
        edge = torch.cat((src,dst),1).to(h.device) #[M,2,D] [num_edges,2,node_features]
        # print("edge shape:", edge.shape)
        lr_e_local = self.edge_proj(edge).squeeze(1)#[M,D]
        # print('lr_e_local shape:', lr_e_local.shape)
        lr_e_local = self.edge_proj2(lr_e_local)  #[num_edges,hidden_dim]
        # print('lr_e_local shape:', lr_e_local.shape)

        hs = []
        gs = dgl.unbatch(g)
        # print("Graph g :", g)
        # print("Graph gs:", gs)
        mask0 = torch.zeros([len(gs),self.max_node_num]).to(h.device) #[1, num_nodes]
        for i,g0 in enumerate(gs):
            Ng = g0.number_of_nodes()
            padding = nn.ConstantPad1d((0,self.max_node_num - Ng),0)
            pad_h = padding(g0.ndata['feature'].T).T #[Nmax, D]
            hs.append(pad_h.unsqueeze(0))
            mask0[i,:Ng] = 1
        hs = torch.cat(hs,0).to(h.device) #[B,Nmax,Din]
        # print("vertex feature shape:", hs.shape)
        hs = self.edge_proj3(hs) #[B,Nmax,hidden_num]
        # print("vertex feature shape 2:", hs.shape)

        if e is None:
            e = torch.ones([g.number_of_edges() ,h.shape[-1]]).to(h.device)
        # Gated-GCN for extract global feature
        hs2g = h
        # print("hs2g shape :", hs2g.shape)
        # print("e :",e.shape)
        # print("global layers :", self.global_layers)
        # print("length of global layers:", len(self.global_layers))
        for conv in self.global_layers:
            # print("conv :", conv)
            hs2g, _ = conv(g, hs2g, e)
        g.ndata['hs2g'] = hs2g
        g.ndata['feature']=hs2g
        # print("hs2g shape:", hs2g.shape)
        global_g = dgl.mean_nodes(g, 'hs2g') #[B,hidden_num]
        # print("shape of global layer:", global_g.shape)

        '''
        # Transformer for extract global feature
        mask_t = mask0.unsqueeze(1)*mask0.unsqueeze(2)
        mask_t = (mask_t==0)
        #mask_t = None

        hs2g = hs.permute((1,0,2))
        for conv in self.global_layers:
            hs2g = conv(hs2g, mask_t)
        global_g = hs2g.permute((1,0,2)).mean(1) #[B,D]
        '''
        # hs ([B, MaxnumNode, Hidden_Num])
        # global_g ([B, Hidden_Num])
        # print("hs shape:", hs.shape)
        edge = self.CrossT(hs, global_g, mask0) #[B,N,N,D]
        # print("edge shape :", edge.shape)

        index_edge = []
        for i,g0 in enumerate(gs):
            index_edge.append(edge[i, g0.all_edges()[0],g0.all_edges()[1],:])
        index_edge = torch.cat(index_edge,0)

        lr_e_global = self.edge_proj4(index_edge)

        if e is not None:
            e = e + lr_e_local + lr_e_global
        else:
            e = lr_e_local + lr_e_global
#        lr_e = lr_e_local + lr_e_global
        # bn=>relu=>dropout
        # print("edge matrix shape :", e.shape)
        e = self.bn_node_lr_e(e)
        e = F.relu(e)
        e = F.dropout(e, 0.1, training=self.training)
        # print("edge matrix shape 2 :", e.shape)
        g.edata['feature']=e
        # print("graph in MEFG module:", g)
        # return e
        return g

# Graph Formation and Graph Modification

## Functions for Graph Generation

In [ ]:
'''
Defining Labels from values as per distribution
values   labels
0-2         0
3-4         1
5-19        2
'''
labels=[]
values=[13,4,19,14,3,3,2,4,7,1,5,4,0,0,3,4,2,13,8,0,2,2,1,6,3,2,3,4,2,4,2,0,7,1,13,2]
for i in values:
  if (i<=2):
        labels.append(0)
  elif (i>=5):
        labels.append(2)
  else :
        labels.append(1)
y_train=torch.tensor(labels)
# print(y_train)
# print(len(set(labels)))

In [ ]:
# Function to create a fully connected graph, defining edge and node features, and apply TTP module on it
def create_basic_graph(df_spdata, num_nodes):
  nx_G = nx.complete_graph(num_nodes)
  g = dgl.from_networkx(nx_G)
  # Adding node fatures to each node from the dataframe containing spectral representaion of each feature
  node_features = torch.tensor(df_spdata.values, dtype=torch.float32)
  # Set node features in the DGL graph
  g.ndata['feature'] = node_features
  g.edata['feature'] = torch.rand(g.num_edges(), 1)
  batch_graphs = g
  batch_x = batch_graphs.ndata['feature']
  batch_e = batch_graphs.edata['feature']
  # print('Graph before TTP',batch_graphs)
  ttp = TTP(in_dim = 120, hidden_dim = 240, edge_thresh=0.2)
  lr_g = ttp(batch_graphs, batch_x, batch_e)
  return lr_g

In [ ]:
# Function to apply MEFG module on a TTP graph
def apply_MEFG(g, num_nodes):
  batch_graphs = g
  batch_x = batch_graphs.ndata['feature']
  batch_e = batch_graphs.edata['feature']
  # print("batch_e shape:", batch_e.shape)
  # batch_x = increase_dim(batch_graphs,batch_x)
  mefg=MEFG(in_dim=120,hidden_dim=240, max_node_num= num_nodes, global_layer_num = 1, dropout = 0.1)
  # e_afterMEFG = mefg(batch_graphs, batch_x, batch_e)
  # return e_afterMEFG
  graph_afterMEFG = mefg(batch_graphs, batch_x, batch_e)
  return graph_afterMEFG

## Graph for Video Data:

In [ ]:
# ### Video
#  Late Function to create list of graphs obatined after application of TTP and MEFG module sequentially on a Fully connected graph
G_list_MEFG_V=[]
num_nodes = 31
def create_graph_list_V(task_dir,csv_files):
  i=0
  for file in csv_files:
    if i< 37:
      df_file=generate_spectral_heatmap(task_dir,file, Primitive_num = num_nodes, K = 60, fre_resolution = 128)
      graph_afterTTP= create_basic_graph(df_file, num_nodes)
      # print("TTP module output before MEFG:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      # e_afterMEFG=apply_MEFG(graph_afterTTP)
      graph_afterMEFG=apply_MEFG(graph_afterTTP, num_nodes)
      # print(" MEFG module ouput graph multi-dimensional edges shape:", e_afterMEFG.shape)
      # print("TTP module output after MEFG:",graph_afterTTP)
      # print("TTP module output after MEFG:",graph_afterMEFG)
      # graph_afterTTP.edata['feature']= e_afterMEFG
      # print("TTP module output after MEFG 2:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      G_list_MEFG_V.append(graph_afterMEFG)
      i+=1
      print(i)

# Defining Task directory and creating list of graphs
task_dir_V="/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_2_h"
task_csv_files_V=os.listdir(task_dir_V)
task_csv_files_V.sort()
create_graph_list_V(task_dir_V,task_csv_files_V)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [ ]:
# # Generated list containing graph data
# print(G_list_MEFG_V)
# print('size of list is:',len(G_list_MEFG_V))

In [ ]:
'''
Create a dataset by converting each graph into PyG Data Object
node features                 x
edge Adjacency Matrix    edge_index
edge features             edge_attr
labels                        y
'''
full_dataset = []
n = 36
for idx in range(0 ,n):
  #edge_index_COO = torch.sparse_coo_tensor(indices=torch.stack([G_list_MEFG[idx].all_edges()[0], G_list_MEFG[idx].all_edges()[1]], dim =0), values=torch.ones_like(G_list_MEFG[idx].all_edges()[0]), size=(31, 31))
  data=Data(x=G_list_MEFG_V[idx].ndata['feature'],  edge_index=torch.stack([G_list_MEFG_V[idx].all_edges()[0], G_list_MEFG_V[idx].all_edges()[1]], dim=0) ,edge_attr=G_list_MEFG_V[idx].edata['feature'],y=y_train[idx])
  full_dataset.append(data)
full_dataset

[Data(x=[31, 240], edge_index=[2, 961], edge_attr=[961, 240], y=2),
 Data(x=[31, 240], edge_index=[2, 934], edge_attr=[934, 240], y=1),
 Data(x=[31, 240], edge_index=[2, 933], edge_attr=[933, 240], y=2),
 Data(x=[31, 240], edge_index=[2, 932], edge_attr=[932, 240], y=2),
 Data(x=[31, 240], edge_index=[2, 961], edge_attr=[961, 240], y=1),
 Data(x=[31, 240], edge_index=[2, 961], edge_attr=[961, 240], y=1),
 Data(x=[31, 240], edge_index=[2, 961], edge_attr=[961, 240], y=0),
 Data(x=[31, 240], edge_index=[2, 961], edge_attr=[961, 240], y=1),
 Data(x=[31, 240], edge_index=[2, 933], edge_attr=[933, 240], y=2),
 Data(x=[31, 240], edge_index=[2, 932], edge_attr=[932, 240], y=0),
 Data(x=[31, 240], edge_index=[2, 932], edge_attr=[932, 240], y=2),
 Data(x=[31, 240], edge_index=[2, 933], edge_attr=[933, 240], y=1),
 Data(x=[31, 240], edge_index=[2, 947], edge_attr=[947, 240], y=0),
 Data(x=[31, 240], edge_index=[2, 932], edge_attr=[932, 240], y=0),
 Data(x=[31, 240], edge_index=[2, 932], edge_att

## Graph for Audio Data

In [ ]:
# Function to create list of graphs obatined after application of TTP and MEFG module sequentially on a Fully connected graph
G_list=[]
G_list_MEFG_A=[]
num_nodes = 91
def create_graph_list_A(task_dir,csv_files):
  i=0
  for file in csv_files:
    if i< 37:
      df_file=generate_spectral_heatmap(task_dir,file, Primitive_num = num_nodes, K = 60, fre_resolution = 128)
      graph_afterTTP= create_basic_graph(df_file, num_nodes)
      print("TTP module output before MEFG:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      # e_afterMEFG=apply_MEFG(graph_afterTTP)
      graph_afterMEFG=apply_MEFG(graph_afterTTP, num_nodes)
      # print(" MEFG module ouput graph multi-dimensional edges shape:", e_afterMEFG.shape)
      # print("TTP module output after MEFG:",graph_afterTTP)
      print("TTP module output after MEFG:",graph_afterMEFG)
      # graph_afterTTP.edata['feature']= e_afterMEFG
      # print("TTP module output after MEFG 2:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      G_list_MEFG_A.append(graph_afterMEFG)
      i+=1
      print(i)

# Defining Task directory and creating list of graphs
task_dir_A="/content/drive/MyDrive/Cambridge_Data/Modify_Audio_Data/Task5p_A"
task_csv_files_A=os.listdir(task_dir_A)
task_csv_files_A.sort()
create_graph_list_A(task_dir_A,task_csv_files_A)

TTP module output before MEFG: Graph(num_nodes=91, num_edges=8192,
      ndata_schemes={'feature': Scheme(shape=(120,), dtype=torch.float32)}
      edata_schemes={'feature': Scheme(shape=(1,), dtype=torch.float32)})
TTP module output after MEFG: Graph(num_nodes=91, num_edges=8192,
      ndata_schemes={'feature': Scheme(shape=(240,), dtype=torch.float32), 'h': Scheme(shape=(240,), dtype=torch.float32), 'Ah': Scheme(shape=(240,), dtype=torch.float32), 'Bh': Scheme(shape=(240,), dtype=torch.float32), 'Dh': Scheme(shape=(240,), dtype=torch.float32), 'Eh': Scheme(shape=(240,), dtype=torch.float32), 'sum_sigma_h': Scheme(shape=(240,), dtype=torch.float32), 'sum_sigma': Scheme(shape=(240,), dtype=torch.float32), 'hs2g': Scheme(shape=(240,), dtype=torch.float32)}
      edata_schemes={'feature': Scheme(shape=(240,), dtype=torch.float32), 'src': Scheme(shape=(120,), dtype=torch.float32), 'dst': Scheme(shape=(120,), dtype=torch.float32), 'e': Scheme(shape=(240,), dtype=torch.float32), 'DEh': Sche

In [ ]:
# Generated list containing graph data
print(G_list_MEFG_A)
print('size of list is:',len(G_list_MEFG_A))

In [ ]:
'''
Create a dataset by converting each graph into PyG Data Object
node features                 x
edge Adjacency Matrix    edge_index
edge features             edge_attr
labels                        y
'''
full_dataset = []
n = 36
for idx in range(0 ,n):
  #edge_index_COO = torch.sparse_coo_tensor(indices=torch.stack([G_list_MEFG[idx].all_edges()[0], G_list_MEFG[idx].all_edges()[1]], dim =0), values=torch.ones_like(G_list_MEFG[idx].all_edges()[0]), size=(31, 31))
  data=Data(x=G_list_MEFG_A[idx].ndata['feature'],  edge_index=torch.stack([G_list_MEFG_A[idx].all_edges()[0], G_list_MEFG_A[idx].all_edges()[1]], dim=0) ,edge_attr=G_list_MEFG_A[idx].edata['feature'],y=y_train[idx])
  full_dataset.append(data)
full_dataset

## Graph for Early fusion

In [ ]:
# Early fusion
G_list=[]
G_list_MEFG_Early=[]
num_nodes = 122
def create_graph_list_EarlyFusion(task_dir_V,task_csv_files_V, task_dir_A,task_csv_files_A):
  i=0
  for i in range (36):
      df_file_V=generate_spectral_heatmap(task_dir_V,task_csv_files_V[i], Primitive_num = 31, K = 60, fre_resolution = 128)
      df_file_A=generate_spectral_heatmap(task_dir_A,task_csv_files_A[i], Primitive_num = 31, K = 60, fre_resolution = 128)
      df_concat = pd.concat([df_file_V, df_file_A], axis=0)
      print(df_concat.shape)
      graph_afterTTP= create_basic_graph(df_concat, num_nodes)
      print("TTP module output before MEFG:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      # e_afterMEFG=apply_MEFG(graph_afterTTP)
      graph_afterMEFG=apply_MEFG(graph_afterTTP, num_nodes)
      # print(" MEFG module ouput graph multi-dimensional edges shape:", e_afterMEFG.shape)
      # print("TTP module output after MEFG:",graph_afterTTP)
      print("TTP module output after MEFG:",graph_afterMEFG)
      # graph_afterTTP.edata['feature']= e_afterMEFG
      # print("TTP module output after MEFG 2:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      G_list_MEFG_Early.append(graph_afterMEFG)
      i+=1
      print(i)

# Defining Task directory and creating list of graphs
task_dir_V="/content/drive/MyDrive/Cambridge_Data/Modify_Data/Task_2_s"
task_dir_A="/content/drive/MyDrive/Cambridge_Data/Modify_Audio_Data/Task2s_A"
task_csv_files_V=os.listdir(task_dir_V)
task_csv_files_A=os.listdir(task_dir_A)
create_graph_list_EarlyFusion(task_dir_V,task_csv_files_V, task_dir_A,task_csv_files_A)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Cambridge_Data/Modify_Audio_Data/Task2s_A'

In [ ]:
# Generated list containing graph data
print(G_list_MEFG_Early)
print('size of list is:',len(G_list_MEFG_Early))

In [ ]:
'''
Create a dataset by converting each graph into PyG Data Object
node features                 x
edge Adjacency Matrix    edge_index
edge features             edge_attr
labels                        y
'''
full_dataset = []
n = 36
for idx in range(0 ,n):
  #edge_index_COO = torch.sparse_coo_tensor(indices=torch.stack([G_list_MEFG[idx].all_edges()[0], G_list_MEFG[idx].all_edges()[1]], dim =0), values=torch.ones_like(G_list_MEFG[idx].all_edges()[0]), size=(31, 31))
  data=Data(x=G_list_MEFG_Early[idx].ndata['feature'],  edge_index=torch.stack([G_list_MEFG_Early[idx].all_edges()[0], G_list_MEFG_Early[idx].all_edges()[1]], dim=0) ,edge_attr=G_list_MEFG_Early[idx].edata['feature'],y=y_train[idx])
  full_dataset.append(data)
full_dataset

## Graph for Late Fusion

In [ ]:
# Defining Task directory and creating list of graphs
task_dir_V="/content/drive/MyDrive/Cambridge_Data/Modify_Video_Data/Task_5"
task_dir_A="/content/drive/MyDrive/Cambridge_Data/Modify_Audio_Data/Task5p_A"
task_csv_files_V=os.listdir(task_dir_V)
task_csv_files_A=os.listdir(task_dir_A)
task_csv_files_V.sort()
task_csv_files_A.sort()

In [ ]:
# ### Video
#  Late Function to create list of graphs obatined after application of TTP and MEFG module sequentially on a Fully connected graph
G_list_MEFG_V=[]
num_nodes = 31
def create_graph_list_V(task_dir,csv_files):
  i=0
  for file in csv_files:
    if i< 37:
      df_file=generate_spectral_heatmap(task_dir,file, Primitive_num = num_nodes, K = 60, fre_resolution = 128)
      graph_afterTTP= create_basic_graph(df_file, num_nodes)
      print("TTP module output before MEFG:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      # e_afterMEFG=apply_MEFG(graph_afterTTP)
      graph_afterMEFG=apply_MEFG(graph_afterTTP, num_nodes)
      # print(" MEFG module ouput graph multi-dimensional edges shape:", e_afterMEFG.shape)
      # print("TTP module output after MEFG:",graph_afterTTP)
      print("TTP module output after MEFG:",graph_afterMEFG)
      # graph_afterTTP.edata['feature']= e_afterMEFG
      # print("TTP module output after MEFG 2:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      G_list_MEFG_V.append(graph_afterMEFG)
      i+=1
      print(i)

create_graph_list_V(task_dir_V,task_csv_files_V)

In [ ]:
# Generated list containing graph data
print(G_list_MEFG_V)
print('size of list is:',len(G_list_MEFG_V))

In [ ]:
'''
Create a dataset by converting each graph into PyG Data Object
node features                 x
edge Adjacency Matrix    edge_index
edge features             edge_attr
labels                        y
'''
full_dataset_V = []
n = 36
for idx in range(0 ,n):
  #edge_index_COO = torch.sparse_coo_tensor(indices=torch.stack([G_list_MEFG[idx].all_edges()[0], G_list_MEFG[idx].all_edges()[1]], dim =0), values=torch.ones_like(G_list_MEFG[idx].all_edges()[0]), size=(31, 31))
  data=Data(x=G_list_MEFG_V[idx].ndata['feature'],  edge_index=torch.stack([G_list_MEFG_V[idx].all_edges()[0], G_list_MEFG_V[idx].all_edges()[1]], dim=0) ,edge_attr=G_list_MEFG_V[idx].edata['feature'],y=y_train[idx])
  full_dataset_V.append(data)
full_dataset_V

In [ ]:
# ## Audio
# Late Function to create list of graphs obatined after application of TTP and MEFG module sequentially on a Fully connected graph
G_list_MEFG_A=[]
num_nodes = 91
def create_graph_list_A(task_dir,csv_files):
  i=0
  for file in csv_files:
    if i< 37:
      df_file=generate_spectral_heatmap(task_dir,file, Primitive_num = num_nodes, K = 60, fre_resolution = 128)
      graph_afterTTP= create_basic_graph(df_file, num_nodes)
      print("TTP module output before MEFG:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      # e_afterMEFG=apply_MEFG(graph_afterTTP)
      graph_afterMEFG=apply_MEFG(graph_afterTTP, num_nodes)
      # print(" MEFG module ouput graph multi-dimensional edges shape:", e_afterMEFG.shape)
      # print("TTP module output after MEFG:",graph_afterTTP)
      print("TTP module output after MEFG:",graph_afterMEFG)
      # graph_afterTTP.edata['feature']= e_afterMEFG
      # print("TTP module output after MEFG 2:",graph_afterTTP)
      # G_list.append(graph_afterTTP)
      G_list_MEFG_A.append(graph_afterMEFG)
      i+=1
      print(i)

create_graph_list_A(task_dir_A,task_csv_files_A)

In [ ]:
# Generated list containing graph data
print(G_list_MEFG_A)
print('size of list is:',len(G_list_MEFG_A))

In [ ]:
'''
Create a dataset by converting each graph into PyG Data Object
node features                 x
edge Adjacency Matrix    edge_index
edge features             edge_attr
labels                        y
'''
full_dataset_A = []
n = 36
for idx in range(0 ,n):
  #edge_index_COO = torch.sparse_coo_tensor(indices=torch.stack([G_list_MEFG[idx].all_edges()[0], G_list_MEFG[idx].all_edges()[1]], dim =0), values=torch.ones_like(G_list_MEFG[idx].all_edges()[0]), size=(31, 31))
  data=Data(x=G_list_MEFG_A[idx].ndata['feature'],  edge_index=torch.stack([G_list_MEFG_A[idx].all_edges()[0], G_list_MEFG_A[idx].all_edges()[1]], dim=0) ,edge_attr=G_list_MEFG_A[idx].edata['feature'],y=y_train[idx])
  full_dataset_A.append(data)
full_dataset_A

In [ ]:
# print(G_list_MEFG[0].edata['feature'])
# print(G_list_MEFG[0].edata['feature'].shape)
# print(G_list_MEFG[0].ndata['feature'].shape)
# print(len(G_list_MEFG))
# # print(G_list_MEFG[0].nodes())
# # print(G_list_MEFG[0].edges())
# print("source nodes",len(G_list_MEFG[0].all_edges()[0]))
# print("destibation nodes",len(G_list_MEFG[0].all_edges()[1]))

In [ ]:
# indices=torch.stack([G_list[0].all_edges()[0], G_list[0].all_edges()[1]] , dim = 1 )
# values=torch.ones_like(G_list[0].all_edges()[0] )

#edge_index_COO = sparse.cat(indices , values )
#edge_index_COO = torch.sparse_coo_tensor(indices=G_list[0].all_edges()[0], values=torch.ones_like(G_list[0].all_edges()[0]), size=(31, 31) )
#edge_index_COO = torch.sparse_coo_tensor(indices=torch.stack([G_list[0].all_edges()[0], G_list[0].all_edges()[1]] , dim =0 ), values=torch.ones_like(G_list[0].all_edges()[0] ), size=(31, 31) )
# edge_index_COO=to_torch_coo_tensor(G_list[0].all_edges())
# edge_index = torch.stack([G_list[0].all_edges()[0], G_list[0].all_edges()[1]], dim=0)
# edge_index_coo = torch.sparse_coo_tensor(edge_index.t(), torch.ones_like(G_list[0].all_edges()[0]), (36, 36))
# edge_index_COO = torch.sparse_coo_tensor(G_list[0].all_edges()[0],G_list[0].all_edges()[1], (932,932))
#print(edge_index_COO)
#data=Data(x=G_list[0].ndata['feature'],  edge_index=edge_index_COO ,edge_attr=G_list[0].edata['feature'],y=y_train[0])

# Graphs Models

# GCN

In [ ]:
# Defining GCN class

class GCN(torch.nn.Module):
    def __init__(self, node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        # self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, num_classes)

    def forward(self, data, dropout_rate):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=dropout_rate, training=self.training)
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, p=dropout_rate, training=self.training)
        x = self.conv2(x, edge_index).relu()
        # x = F.dropout(x, p=dropout_rate, training=self.training)
        # x = self.conv3(x, edge_index).relu()
        x = global_mean_pool(x, batch=data.batch)
        x = self.lin(x)

        return F.softmax(x, dim=1)

# model = GCN(hidden_channels)
# print(model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device', device)

## Training loop for GCN

Same training loop is for Video, Audio and Early Fusion. Different for Late fusion only

In [ ]:
def train_epoch(train_loader, dr, optimizer, criterion, model_GCN, scheduler):
  loss_train_per = []
  accuracy_train = []
  model_GCN.train()
  for data in train_loader :
        optimizer.zero_grad()
        output_train =  model_GCN(data, dropout_rate= dr)
        target_train = data.y  # Assuming your target labels are stored in 'y'
        loss = criterion(output_train, target_train)
        loss.backward(retain_graph= True)
        optimizer.step()
        predicted_train = output_train.argmax(dim=1)
        accuracy_train.append(accuracy_score(target_train.cpu().numpy(), predicted_train.cpu().numpy()))
        loss_train_per.append(loss.item())
  loss_train_total= np.mean(loss_train_per)
  train_accuracy= np.mean(accuracy_train)
  if scheduler is not None:
        scheduler.step()  # Update learning rate
  return train_accuracy, loss_train_total,model_GCN, optimizer

In [ ]:
def valid_epoch(test_loader, dr, criterion,model_GCN):
  val_loss_per=[]
  accuracy_test=[]
  model_GCN.eval()
  with torch.no_grad():
      for data in test_loader:
          output_test =  model_GCN(data, dropout_rate= dr)
          predicted_test = output_test.argmax(dim=1)
          target_test = data.y
          accuracy_test.append(accuracy_score(target_test.cpu().numpy(), predicted_test.cpu().numpy()))
          loss = criterion(output_test, target_test)
          val_loss_per.append(loss.item())
  val_loss_total= np.mean(val_loss_per)
  val_accuracy= np.mean(accuracy_test)
  return val_accuracy, val_loss_total

In [ ]:
def train_valid_loop_ADAM(lr, dr, hidden_classes, trial_subdir, plot_subdir, task_name):
    num_epochs = 20
    best_epoch = -1
    best_stop_epoch=-1
    early_stop_count = 0
    best_val_loss = float('inf')
    best_val_acc = 0
    patience = 3
    batch_size = 1
    all_train_losses = [[] for _ in range(36)]
    all_train_accuracies = [[] for _ in range(36)]
    avg_train_losses=[] # Average of all training losses across all subjects per epoch
    avg_train_accuracies=[] # Average of all training accuracies across all subjects per epoch
    all_val_losses = []
    all_val_accuracies = []

    # GCN MODEL with 2 layers in architecture
    model_GCN = [ GCN(node_features= 240,hidden_channels=hidden_classes, num_classes=3).to(device) for _ in range(36)]
    optimizer = [torch.optim.Adam(model_GCN[_].parameters(), lr= lr) for _ in range(36)]
    scheduler = [StepLR(optimizer[_], step_size=1, gamma=0.9) for _ in range(36)] # decrease LR by a factor of 0.9 every epoch
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        epoch_subdir = os.path.join(trial_subdir, f"epoch_{epoch}")
        os.makedirs(epoch_subdir, exist_ok=True)
        # print(f"## epoch {epoch}  ##")
        val_accuracies =[]
        val_losses=[]
        for idx in range(36):
            # DataLoader setup
            # print(f"## Subject {idx} is test subject  ##")
            train_indices = list(range(36))
            del train_indices[idx]
            train_dataset = [full_dataset[i].to(device) for i in train_indices]
            test_dataset = [full_dataset[idx].to(device)]

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            # Training loop
            train_accuracy, train_loss, model_GCN[idx], optimizer[idx] = train_epoch(train_loader, dr, optimizer[idx], criterion, model_GCN[idx], scheduler[idx])
            all_train_losses[idx].append(train_loss)
            all_train_accuracies[idx].append(train_accuracy)

            # Validation loop
            val_accuracy,val_loss = valid_epoch(test_loader, dr, criterion, model_GCN[idx])
            val_losses.append(val_loss)
            val_accuracies.append(val_accuracy)
            model_filename = os.path.join(epoch_subdir, f"subject_{idx}_model.pt")
            torch.save({
                        'model_state_dict': model_GCN[idx].state_dict(),
                        'learning rate': lr,
                        'dropout':dr,
                        'hidden_classes':hidden_classes,
                        }, model_filename)

        all_val_losses.append(np.mean(val_losses))
        all_val_accuracies.append(np.mean(val_accuracies))
        print(f" For epoch {epoch}, Validation Loss: {np.mean(val_losses):.4f}, Validation Accuracy: {np.mean(val_accuracies):.4f}")
        # Early stopping
        if np.mean(val_losses) < best_val_loss:
            best_val_loss = np.mean(val_losses)
            early_stop_count = 0
        else:
            early_stop_count += 1

        if np.mean(val_accuracies) > best_val_acc:
            best_val_acc= np.mean(val_accuracies)
            delete_dir_path= os.path.join(trial_subdir, f"epoch_{best_epoch}")
            if os.path.exists(delete_dir_path):
                  shutil.rmtree(delete_dir_path)
            best_epoch=epoch
        else:
            delete_dir_path= os.path.join(trial_subdir, f"epoch_{epoch}")
            if os.path.exists(delete_dir_path):
                  shutil.rmtree(delete_dir_path)

        if early_stop_count >= patience:
            print(f'Early stopped at epoch: {epoch}')
            best_stop_epoch= epoch
            print(f"Final Validation Accurcay: {best_val_acc}")
            break

    # Calculating Average of all training losses across all subjects per epoch
    for i in range(best_stop_epoch+1):
      avg_train_losses.append(np.average([x[i] for x in all_train_losses]))
      avg_train_accuracies.append(np.average([x[i] for x in all_train_accuracies]))

#     print(f'length of average train losses, {len(avg_train_losses[0])}')
        # Plotting code for all subjects
    fig_loss, axs_loss = plt.subplots(6, 6, figsize=(18, 18))
    fig_acc, axs_acc = plt.subplots(6, 6, figsize=(18, 18))

    for idx in range(36):
        # Plotting Loss
        # epochs = range(len(all_train_losses[idx]))
        ax_loss = axs_loss[idx // 6, idx % 6]
        ax_loss.plot(range(len(all_train_losses[idx])), all_train_losses[idx], label='Train', color='red')
        ax_loss.set_title(f"Loss - Subject {idx}-{task_name}")
        ax_loss.legend()

        # Plotting Accuracy
        # epochs = range(len(all_train_accuracies[idx]))
        ax_acc = axs_acc[idx // 6, idx % 6]
        ax_acc.plot(range(len(all_train_accuracies[idx])), all_train_accuracies[idx], label='Train', color='green')
        ax_acc.set_title(f"Accuracy - Subject {idx}-{task_name}")
        ax_acc.legend()
        # Set integer labels on x-axis
        ax_loss.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
        ax_acc.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))

    # Adjust layout
    fig_loss.tight_layout()
    fig_acc.tight_layout()

    # Validation loss and Validation accuracy curves vs. epoch
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
    ax1.plot(range(len(all_val_losses)), all_val_losses, label='Validation Loss', color='orange')
    ax1.plot(range(len(avg_train_losses)), avg_train_losses, label='Training Loss', color='blue')
    ax1.set_title('Train + Val Loss vs. Epochs')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax2.plot(range(len(all_val_accuracies)), all_val_accuracies, label='Validation Accuracy', color='orange')
    ax2.plot(range(len(avg_train_accuracies)), avg_train_accuracies, label='Training Accuracy', color='blue')
    ax2.set_title('Train + Val Accuracy vs. Epochs')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
    ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
    plt.tight_layout()

    #  Saving the plots
    fig_loss.savefig(os.path.join(plot_subdir+"/"+f"training_loss.png"))
    fig_acc.savefig(os.path.join(plot_subdir+ "/"+ f'training_accuracy.png'))
    fig.savefig(os.path.join(plot_subdir+ "/"+ f'validation_plots.png'))

    # Close the figures
    # plt.close(fig_loss)
    # plt.close(fig_acc)
    # plt.close(fig)

    # Show plots
    plt.show()
    print("Final validation accuracy :", best_val_acc)
    print("Epoch at which Best model with highest validation accuracy is obtained:", best_epoch)
    print("Epochs at which Early stopping is done:", best_stop_epoch)
    return best_val_acc

## Video Data

Training using Optuna for hyperparameter tuning on video data

In [ ]:
best_trial_dir = None
best_trial_acc = float('-inf')
def objective_ADAM(trial):
    global best_trial_dir, best_trial_acc
    # Hyperparams to tune
    task_name = 'Task5_V'
    lr = trial.suggest_float("lr", 1e-5, 1e-2)
    dr = trial.suggest_float("dropout", 0.1, 0.8)
    hidden_classes = trial.suggest_int('hidden_classes', 16, 180)
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}")
    save_dir=f'/content/GCN_checkpoints_{task_name}/'
    trial_subdir = os.path.join(save_dir, f"trial_{trial.number}")
    os.makedirs(trial_subdir, exist_ok=True)
    plot_dir=f'/content/GCN_plots_{task_name}/'
    plot_subdir = os.path.join(plot_dir, f"trial_{trial.number}")
    os.makedirs(plot_subdir, exist_ok=True)
    # test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads, trial_subdir)
    test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes, trial_subdir, plot_subdir, task_name)
    if test_acc > best_trial_acc:
      best_trial_acc = test_acc
      remove_trial_subdir= os.path.join(save_dir,f"trial_{best_trial_dir}")
      if os.path.exists(remove_trial_subdir):
          shutil.rmtree(remove_trial_subdir)
      best_trial_dir= trial.number
    else:
        if os.path.exists(trial_subdir):
            shutil.rmtree(trial_subdir)
    return test_acc

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed= seed_value))
study.optimize(objective_ADAM, n_trials= 40 )

print(study.best_trial)

To load model from checkpoints and then directly test it for Video data

In [ ]:
batch_size= 1
task_name = 'Task4_P3_V' # Give the name of task for which you are testing results using checkpoints
save_dir="/kaggle/working/GAT_checkpoints_Task5_A/trial_8/epoch_1" # path to directory where checkpoints are saved
total_correct = 0
total_samples = 0
accuracy_list = []
# Perform 1-subject leave-out cross-validation
for idx in range(36):
    # Load the checkpoint
    checkpoint_path = os.path.join(save_dir, f"subject_{idx}_model.pt")
    checkpoint = torch.load(checkpoint_path)

    # Retrieve parameters
    model_state_dict = checkpoint['model_state_dict']
    lr = checkpoint['learning rate']
    dr = checkpoint['dropout']
    hidden_classes = checkpoint['hidden_classes']
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}")

    model_GCN = GCN(node_features= 240,hidden_channels= hidden_classes, num_classes=3).to(device)
    # Load the model weights
    model_GCN.load_state_dict(model_state_dict)
    model_GCN.eval()  # Set the model to evaluation mode

    # Create DataLoader for the test dataset
    test_dataset = [full_dataset[idx].to(device)]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Testing loop
    with torch.no_grad():
        for data in test_loader:
            output_test = model_GCN(data, dropout_rate=dr)
            predicted_test = output_test.argmax(dim=1)
            target_test = data.y
            print("Test Subject:",idx,"Predicted label:", predicted_test, "True label:", target_test)
            total_correct += (predicted_test == target_test).sum().item()
            total_samples += data.y.size(0)
            accuracy_list.append(accuracy_score(target_test.cpu().numpy(), predicted_test.cpu().numpy()))

# Calculate and print test accuracy for the current subject
test_accuracy = total_correct / total_samples
print("total correct :", total_correct," total samples :", total_samples)
print(f"Using Subject {idx} as test, Test Accuracy: {test_accuracy:.4f}")
print("Accuracy using inbuilt function:", np.mean(accuracy_list))

## Audio Data

Training using Optuna for hyperparameter tuning on Audio data

In [ ]:
best_trial_dir = None
best_trial_acc = float('-inf')
def objective_ADAM(trial):
    global best_trial_dir, best_trial_acc
    # Hyperparams to tune
    task_name = 'Task5_A'
    lr = trial.suggest_float("lr", 1e-5, 1e-2)
    dr = trial.suggest_float("dropout", 0.1, 0.8)
    hidden_classes = trial.suggest_int('hidden_classes', 16, 180)
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}")
    save_dir=f'/content/GCN_checkpoints_{task_name}/'
    trial_subdir = os.path.join(save_dir, f"trial_{trial.number}")
    os.makedirs(trial_subdir, exist_ok=True)
    plot_dir=f'/content/GCN_plots_{task_name}/'
    plot_subdir = os.path.join(plot_dir, f"trial_{trial.number}")
    os.makedirs(plot_subdir, exist_ok=True)
    # test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads, trial_subdir)
    test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes, trial_subdir, plot_subdir, task_name)
    if test_acc > best_trial_acc:
      best_trial_acc = test_acc
      remove_trial_subdir= os.path.join(save_dir,f"trial_{best_trial_dir}")
      if os.path.exists(remove_trial_subdir):
          shutil.rmtree(remove_trial_subdir)
      best_trial_dir= trial.number
    else:
        if os.path.exists(trial_subdir):
            shutil.rmtree(trial_subdir)
    return test_acc

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed= seed_value))
study.optimize(objective_ADAM, n_trials= 40 )

print(study.best_trial)

To load model from checkpoints and then directly test it for audio data

In [ ]:
batch_size= 1
task_name = 'Task4_P3_A' # Give the name of task for which you are testing results using checkpoints
save_dir="/kaggle/working/GAT_checkpoints_Task5_A/trial_8/epoch_1" # path to directory where checkpoints are saved
total_correct = 0
total_samples = 0
accuracy_list = []
# Perform 1-subject leave-out cross-validation
for idx in range(36):
    # Load the checkpoint
    checkpoint_path = os.path.join(save_dir, f"subject_{idx}_model.pt")
    checkpoint = torch.load(checkpoint_path)

    # Retrieve parameters
    model_state_dict = checkpoint['model_state_dict']
    lr = checkpoint['learning rate']
    dr = checkpoint['dropout']
    hidden_classes = checkpoint['hidden_classes']
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}")

    model_GCN = GCN(node_features= 240,hidden_channels=num_channels, num_classes=3).to(device)
    # Load the model weights
    model_GCN.load_state_dict(model_state_dict)
    model_GCN.eval()  # Set the model to evaluation mode

    # Create DataLoader for the test dataset
    test_dataset = [full_dataset[idx].to(device)]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Testing loop
    with torch.no_grad():
        for data in test_loader:
            output_test = model_GCN(data, dropout_rate=dr)
            predicted_test = output_test.argmax(dim=1)
            target_test = data.y
            print("Test Subject:",idx,"Predicted label:", predicted_test, "True label:", target_test)
            total_correct += (predicted_test == target_test).sum().item()
            total_samples += data.y.size(0)
            accuracy_list.append(accuracy_score(target_test.cpu().numpy(), predicted_test.cpu().numpy()))

# Calculate and print test accuracy for the current subject
test_accuracy = total_correct / total_samples
print("total correct :", total_correct," total samples :", total_samples)
print(f"Using Subject {idx} as test, Test Accuracy: {test_accuracy:.4f}")
print("Accuracy using inbuilt function:", np.mean(accuracy_list))

## Early Fusion

Training using Optuna for hyperparameter tuning on Early Fusio of video and Audio data

In [ ]:
best_trial_dir = None
best_trial_acc = float('-inf')
def objective_ADAM(trial):
    global best_trial_dir, best_trial_acc
    # Hyperparams to tune
    task_name = 'Task5_E'
    lr = trial.suggest_float("lr", 1e-5, 1e-2)
    dr = trial.suggest_float("dropout", 0.1, 0.8)
    hidden_classes = trial.suggest_int('hidden_classes', 16, 180)
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}")
    save_dir=f'/content/GCN_checkpoints_{task_name}/'
    trial_subdir = os.path.join(save_dir, f"trial_{trial.number}")
    os.makedirs(trial_subdir, exist_ok=True)
    plot_dir=f'/content/GCN_plots_{task_name}/'
    plot_subdir = os.path.join(plot_dir, f"trial_{trial.number}")
    os.makedirs(plot_subdir, exist_ok=True)
    # test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads, trial_subdir)
    test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes, trial_subdir, plot_subdir, task_name)
    if test_acc > best_trial_acc:
      best_trial_acc = test_acc
      remove_trial_subdir= os.path.join(save_dir,f"trial_{best_trial_dir}")
      if os.path.exists(remove_trial_subdir):
          shutil.rmtree(remove_trial_subdir)
      best_trial_dir= trial.number
    else:
        if os.path.exists(trial_subdir):
            shutil.rmtree(trial_subdir)
    return test_acc

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed= seed_value))
study.optimize(objective_ADAM, n_trials= 40 )

print(study.best_trial)

To load model from checkpoints and then directly test it for Early fusion of Video and Audio data

In [ ]:
batch_size= 1
task_name = 'Task4_P3_E' # Give the name of task for which you are testing results using checkpoints
save_dir="/kaggle/working/GAT_checkpoints_Task5_A/trial_8/epoch_1" # path to directory where checkpoints are saved
total_correct = 0
total_samples = 0
accuracy_list = []
# Perform 1-subject leave-out cross-validation
for idx in range(36):
    # Load the checkpoint
    checkpoint_path = os.path.join(save_dir, f"subject_{idx}_model.pt")
    checkpoint = torch.load(checkpoint_path)

    # Retrieve parameters
    model_state_dict = checkpoint['model_state_dict']
    lr = checkpoint['learning rate']
    dr = checkpoint['dropout']
    hidden_classes = checkpoint['hidden_classes']
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}")

    model_GCN = GCN(node_features= 240,hidden_channels=num_channels, num_classes=3).to(device)
    # Load the model weights
    model_GCN.load_state_dict(model_state_dict)
    model_GCN.eval()  # Set the model to evaluation mode

    # Create DataLoader for the test dataset
    test_dataset = [full_dataset[idx].to(device)]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Testing loop
    with torch.no_grad():
        for data in test_loader:
            output_test = model_GCN(data, dropout_rate=dr)
            predicted_test = output_test.argmax(dim=1)
            target_test = data.y
            print("Test Subject:",idx,"Predicted label:", predicted_test, "True label:", target_test)
            total_correct += (predicted_test == target_test).sum().item()
            total_samples += data.y.size(0)
            accuracy_list.append(accuracy_score(target_test.cpu().numpy(), predicted_test.cpu().numpy()))

# Calculate and print test accuracy for the current subject
test_accuracy = total_correct / total_samples
print("total correct :", total_correct," total samples :", total_samples)
print(f"Using Subject {idx} as test, Test Accuracy: {test_accuracy:.4f}")
print("Accuracy using inbuilt function:", np.mean(accuracy_list))

## Late Fusion

For late fusion the training and validation loop are different from previous training and validation loops.

In [ ]:
def train_epoch_late(train_loader1, dr1, optimizer1, model_GCN1, scheduler1, train_loader2, dr2, optimizer2, model_GCN2, scheduler2,criterion):
    loss_train_per = []
    accuracy_train = []
    model_GCN1.train()
    model_GCN2.train()
    for data1, data2 in zip(train_loader1, train_loader2):
        optimizer1.zero_grad()
        output_train1 = model_GCN1(data1, dropout_rate= dr1)
        target_train1 = data1.y
        loss1 = criterion(output_train1, target_train1)
        loss1.backward(retain_graph= True)
        optimizer1.step()

        optimizer2.zero_grad()
        output_train2 = model_GCN2(data2, dropout_rate= dr2)
        target_train2 = data2.y
        loss2 = criterion(output_train2, target_train2)
        loss2.backward(retain_graph= True)
        optimizer2.step()
        final_output = (output_train1+output_train2)/2.0
        predicted_labels = torch.argmax(final_output, dim=1)
        target_fused = torch.max(target_train1, target_train2)
        loss = (loss1.item()+ loss2.item())/2.0
        loss_train_per.append(loss)
        accuracy_train.append(accuracy_score(target_fused.cpu().numpy(), predicted_labels.cpu().numpy()))
    loss_train_total= np.mean(loss_train_per)
    train_accuracy= np.mean(accuracy_train)
    if scheduler1 is not None:
        scheduler1.step()  # Update learning rate
    if scheduler2 is not None:
        scheduler2.step()  # Update learning rate

    return train_accuracy, loss_train_total,model_GCN1, optimizer1, model_GCN2, optimizer2

In [ ]:
def valid_epoch_late(test_loader1, dr1, model_GCN1, test_loader2, dr2, model_GCN2, criterion):
  val_loss_per=[]
  accuracy_test=[]
  model_GCN1.eval()
  model_GCN2.eval()
  with torch.no_grad():
      for data1, data2 in zip(test_loader1, test_loader2):
        output_test1 = model_GCN1(data1, dropout_rate= dr2)
        target_test1 = data1.y
        output_test2 = model_GCN2(data2, dropout_rate= dr2)
        target_test2 = data2.y
        output_test_fused = (output_test1+output_test2)/2.0
        predicted_labels = torch.argmax(output_test_fused, dim=1)
        target_fused = torch.max(target_test1, target_test2)
        accuracy_test.append(accuracy_score(target_fused.cpu().numpy(), predicted_labels.cpu().numpy()))
        loss = criterion(output_test_fused, target_fused)
        val_loss_per.append(loss.item())
  val_loss_total= np.mean(val_loss_per)
  val_accuracy= np.mean(accuracy_test)
  return val_accuracy, val_loss_total

In [ ]:
def train_valid_loop_ADAM_late(lr1, dr1, hidden_classes1, lr2, dr2, hidden_classes2, trial_subdir, plot_subdir, task_name):
    num_epochs = 20
    best_epoch = -1
    best_stop_epoch=-1
    early_stop_count = 0
    best_val_loss = float('inf')
    best_val_acc = 0
    patience = 3
    batch_size = 1
    all_train_losses = [[] for _ in range(36)]
    all_train_accuracies = [[] for _ in range(36)]
    avg_train_losses=[] # Average of all training losses across all subjects per epoch
    avg_train_accuracies=[] # Average of all training accuracies across all subjects per epoch
    all_val_losses = []
    all_val_accuracies = []
    '''GAT MODEL with 2 layers in architecture
    model_GAT = [GAT(node_features=240, hidden_channels= hidden_classes,num_heads=num_heads, num_classes=3, dropout_rate= dr).to(device) for _ in range(36)]
    '''
    # GAT model with parameters to choose no. of layers in architecture
    model_GCN1 = [GCN(node_features= 240,hidden_channels=hidden_classes1, num_classes=3).to(device) for _ in range(36)]
    optimizer1 = [torch.optim.Adam(model_GCN1[_].parameters(), lr= lr1) for _ in range(36)]
    scheduler1 = [StepLR(optimizer1[_], step_size=1, gamma=0.9) for _ in range(36)] # decrease LR by a factor of 0.9 every epoch

    model_GCN2 = [GCN(node_features= 240,hidden_channels=hidden_classes1, num_classes=3).to(device) for _ in range(36)]
    optimizer2 = [torch.optim.Adam(model_GCN2[_].parameters(), lr= lr2) for _ in range(36)]
    scheduler2 = [StepLR(optimizer2[_], step_size=1, gamma=0.9) for _ in range(36)] # decrease LR by a factor of 0.9 every epoch
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        epoch_subdir = os.path.join(trial_subdir, f"epoch_{epoch}")
        os.makedirs(epoch_subdir, exist_ok=True)
        # print(f"## epoch {epoch}  ##")
        val_accuracies =[]
        val_losses=[]
        for idx in range(36):
            # DataLoader setup
            # print(f"## Subject {idx} is test subject  ##")
            train_indices = list(range(36))
            del train_indices[idx]
            train_dataset1 = [full_dataset_V[i].to(device) for i in train_indices]
            test_dataset1 = [full_dataset_V[idx].to(device)]
            train_dataset2 = [full_dataset_A[i].to(device) for i in train_indices]
            test_dataset2 = [full_dataset_A[idx].to(device)]

            train_loader1 = DataLoader(train_dataset1, batch_size=batch_size, shuffle=True)
            test_loader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=False)
            train_loader2 = DataLoader(train_dataset2, batch_size=batch_size, shuffle=True)
            test_loader2 = DataLoader(test_dataset2, batch_size=batch_size, shuffle=False)

            # Training loop
            train_accuracy, train_loss, model_GCN1[idx], optimizer1[idx], model_GCN2[idx], optimizer2[idx] = train_epoch_late(train_loader1, dr1, optimizer1[idx], model_GCN1[idx], scheduler1[idx], train_loader2, dr2, optimizer2[idx], model_GCN2[idx], scheduler2[idx],criterion)
            all_train_losses[idx].append(train_loss)
            all_train_accuracies[idx].append(train_accuracy)

            # Validation loop
            val_accuracy,val_loss = valid_epoch_late(test_loader1, dr1, model_GCN1[idx], test_loader2, dr2, model_GCN2[idx], criterion)
            val_losses.append(val_loss)
            val_accuracies.append(val_accuracy)
            # Saving model checkpoints
            model_filename1 = os.path.join(epoch_subdir, f"Model1_subject_{idx}_model.pt")
            torch.save({
                        'model_state_dict1': model_GCN1[idx].state_dict(),
                        'learning rate1': lr1,
                        'dropout1':dr1,
                        'hidden_classes1':hidden_classes1,
                        }, model_filename1)
            model_filename2 = os.path.join(epoch_subdir, f"Model2_subject_{idx}_model.pt")
            torch.save({
                        'model_state_dict2': model_GCN2[idx].state_dict(),
                        'learning rate2': lr2,
                        'dropout2':dr2,
                        'hidden_classes2':hidden_classes2,
                        }, model_filename2)

        all_val_losses.append(np.mean(val_losses))
        all_val_accuracies.append(np.mean(val_accuracies))
        print(f" For epoch {epoch}, Validation Loss: {np.mean(val_losses):.4f}, Validation Accuracy: {np.mean(val_accuracies):.4f}")

        # Early stopping
        if np.mean(val_losses) < best_val_loss:
            best_val_loss = np.mean(val_losses)
            early_stop_count = 0
        else:
            early_stop_count += 1

        if np.mean(val_accuracies) > best_val_acc:
            best_val_acc= np.mean(val_accuracies)
            delete_dir_path= os.path.join(trial_subdir, f"epoch_{best_epoch}")
            if os.path.exists(delete_dir_path):
                  shutil.rmtree(delete_dir_path)
            best_epoch=epoch
        else:
            delete_dir_path= os.path.join(trial_subdir, f"epoch_{epoch}")
            if os.path.exists(delete_dir_path):
                  shutil.rmtree(delete_dir_path)

        if early_stop_count >= patience:
            print(f'Early stopped at epoch: {epoch}')
            best_stop_epoch= epoch
            print(f"Final Validation Accurcay: {best_val_acc}")
            break

    # Calculating Average of all training losses across all subjects per epoch
    for i in range(best_stop_epoch+1):
      avg_train_losses.append(np.average([x[i] for x in all_train_losses]))
      avg_train_accuracies.append(np.average([x[i] for x in all_train_accuracies]))

#     print(f'length of average train losses, {len(avg_train_losses[0])}')
        # Plotting code for all subjects
    fig_loss, axs_loss = plt.subplots(6, 6, figsize=(18, 18))
    fig_acc, axs_acc = plt.subplots(6, 6, figsize=(18, 18))

    for idx in range(36):
        # Plotting Loss
        # epochs = range(len(all_train_losses[idx]))
        ax_loss = axs_loss[idx // 6, idx % 6]
        ax_loss.plot(range(len(all_train_losses[idx])), all_train_losses[idx], label='Train', color='red')
        ax_loss.set_title(f"Loss - Subject {idx}-{task_name}")
        ax_loss.legend()

        # Plotting Accuracy
        # epochs = range(len(all_train_accuracies[idx]))
        ax_acc = axs_acc[idx // 6, idx % 6]
        ax_acc.plot(range(len(all_train_accuracies[idx])), all_train_accuracies[idx], label='Train', color='green')
        ax_acc.set_title(f"Accuracy - Subject {idx}-{task_name}")
        ax_acc.legend()
        # Set integer labels on x-axis
        ax_loss.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
        ax_acc.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))

    # Adjust layout
    fig_loss.tight_layout()
    fig_acc.tight_layout()

    # Validation loss and Validation accuracy curves vs. epoch
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
    ax1.plot(range(len(all_val_losses)), all_val_losses, label='Validation Loss', color='orange')
    ax1.plot(range(len(avg_train_losses)), avg_train_losses, label='Training Loss', color='blue')
    ax1.set_title('Train + Val Loss vs. Epochs')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax2.plot(range(len(all_val_accuracies)), all_val_accuracies, label='Validation Accuracy', color='orange')
    ax2.plot(range(len(avg_train_accuracies)), avg_train_accuracies, label='Training Accuracy', color='blue')
    ax2.set_title('Train + Val Accuracy vs. Epochs')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
    ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
    plt.tight_layout()

    #  Saving the plots
    fig_loss.savefig(os.path.join(plot_subdir+"/"+f"training_loss.png"))
    fig_acc.savefig(os.path.join(plot_subdir+ "/"+ f'training_accuracy.png'))
    fig.savefig(os.path.join(plot_subdir+ "/"+ f'validation_plots.png'))

    # Show plots
    plt.show()
    print("Final validation accuracy :", best_val_acc)
    print("Epoch at which Best model with highest validation accuracy is obtained:", best_epoch)
    print("Epochs at which Early stopping is done:", best_stop_epoch)
    return best_val_acc

Training using Optuna for hyperparameter tuning on Late fusion of video and Audio data

In [ ]:
best_trial_dir = None
best_trial_acc = float('-inf')
task_name = 'Task5_L'
def objective_ADAM_late(trial):
    global best_trial_dir, best_trial_acc, task_name
    # Hyperparams to tune
    lr1 = trial.suggest_float("lr1", 1e-5, 1e-2)
    lr2 = trial.suggest_float("lr2", 1e-5, 1e-2)
    dr1 = trial.suggest_float("dropout1", 0.1, 0.8)
    dr2 = trial.suggest_float("dropout2", 0.1, 0.8)
    hidden_classes1 = trial.suggest_int("hidden_classes1", 16, 180)
    hidden_classes2 = trial.suggest_int("hidden_classes2", 16, 180)
    print("learning rate 1:", lr1,"learning rate 2:", lr2,"dropout rate1:", dr1, "dropout rate2:", dr2,"hidden_classes1 :", hidden_classes1, "hidden_classes2:", hidden_classes2)
    save_dir=f'/kaggle/working/GCN_checkpoints_{task_name}/'
    trial_subdir = os.path.join(save_dir, f"trial_{trial.number}")
    os.makedirs(trial_subdir, exist_ok=True)
    plot_dir=f'/kaggle/working/GCN_plots_{task_name}/'
    plot_subdir = os.path.join(plot_dir, f"trial_{trial.number}")
    os.makedirs(plot_subdir, exist_ok=True)
    # test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads, trial_subdir)
    test_acc = train_valid_loop_ADAM_late(lr1, dr1, hidden_classes1, lr2, dr2, hidden_classes2, trial_subdir, plot_subdir, task_name)
    if test_acc > best_trial_acc:
      best_trial_acc = test_acc
      remove_trial_subdir= os.path.join(save_dir,f"trial_{best_trial_dir}")
      if os.path.exists(remove_trial_subdir):
          shutil.rmtree(remove_trial_subdir)
      best_trial_dir= trial.number
    else:
        if os.path.exists(trial_subdir):
            shutil.rmtree(trial_subdir)
    return test_acc

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed= seed_value))
study.optimize(objective_ADAM_late, n_trials= 40 )

print(study.best_trial)

To load model from checkpoints and then directly test it for late fusion of video and Audio data

In [ ]:
batch_size=1
task_name = 'Task5_L' # name of task for which you are testing
save_dir="/content/drive/MyDrive/GAT_checkpoints/Late Fusion/trial_23" # path of directory where checkpoints are saved
total_correct = 0
total_samples = 0
test_accuracy = []
# Perform 1-subject leave-out cross-validation
for idx in range(36):
    # Load the checkpoint
    checkpoint_path1 = os.path.join(save_dir, f"Model1_subject_{idx}_model.pt")
    checkpoint1 = torch.load(checkpoint_path1)
    # Retrieve parameters
    model_state_dict1 = checkpoint1['model_state_dict']
    lr1 = checkpoint1['learning rate']
    dr1 = checkpoint1['dropout']
    hidden_classes1 = checkpoint1['hidden_classes']
    print(f"learning rate:{lr1},dropout: {dr1}, hidden_classes: {hidden_classes1}")

    model_GCN1 = GCN(node_features= 240,hidden_channels= hidden_classes1, num_classes=3).to(device)
    # Load the model weights
    model_GCN1.load_state_dict(model_state_dict1)
    model_GCN1.eval()  # Set the model to evaluation mode


    checkpoint_path2 = os.path.join(save_dir, f"Model2_subject_{idx}_model.pt")
    checkpoint2 = torch.load(checkpoint_path2)
    # Retrieve parameters
    model_state_dict2 = checkpoint2['model_state_dict']
    lr2 = checkpoint2['learning rate']
    dr2 = checkpoint2['dropout']
    hidden_classes2 = checkpoint2['hidden_classes']
    print(f"learning rate:{lr2},dropout: {dr2}, hidden_classes: {hidden_classes2}")

    model_GCN2 = GCN(node_features= 240,hidden_channels=hidden_classes2, num_classes=3).to(device)
    # Load the model weights
    model_GCN2.load_state_dict(model_state_dict2)
    model_GCN2.eval()  # Set the model to evaluation mode

    # Create DataLoader for the test dataset
    test_dataset1 = [full_dataset_V[idx].to(device)]
    test_loader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=False)
    test_dataset2 = [full_dataset_A[idx].to(device)]
    test_loader2 = DataLoader(test_dataset2, batch_size=batch_size, shuffle=False)

    # Testing loop
    model_GCN1.eval()
    model_GCN2.eval()
    with torch.no_grad():
        for data1, data2 in zip(test_loader1, test_loader2):
            output_test1 = model_GCN1(data1, dropout_rate= dr1)
            target_test1 = data1.y
            output_test2 = model_GCN2(data2, dropout_rate= dr2)
            target_test2 = data2.y
            final_output = (output_test1+output_test2)/2.0
            predicted_labels = torch.argmax(final_output, dim=1)
            target_fused = torch.max(target_test1, target_test2)
            total_correct += (predicted_labels == target_fused).sum().item()
            total_samples += data1.y.size(0)
            print("Test Subject:",idx," predicted label:", predicted_labels ," True label : ", target_fused)
            test_accuracy.append(accuracy_score(target_fused.cpu().numpy(), predicted_labels.cpu().numpy()))

# Calculate and print test accuracy for the current subject
test_accuracy = total_correct / total_samples
print("total correct :", total_correct," total samples :", total_samples)
print(f"Using Subject {idx} as test, Test Accuracy: {test_accuracy:.4f}")
print("Accuracy using inbuilt function:", np.mean(test_accuracy))

# GAT

GAT Model with 2 layers of GATCONV

In [ ]:
# class GAT(torch.nn.Module):
#     def __init__(self, node_features, hidden_channels,num_heads, num_classes, dropout_rate):
#         super(GAT, self).__init__()
#         self.head = num_heads

#         self.conv1 = GATConv(node_features, hidden_channels, heads=self.head, dropout=dropout_rate)
#         self.conv2 = GATConv(hidden_channels * self.head, hidden_channels, heads=self.head,
#                              dropout=dropout_rate)

#         self.lin = nn.Linear(hidden_channels * self.head, num_classes)

#     def forward(self, data, dropout_rate):
#         x, batch, edge_index, edge_attr = data.x, data.batch, data.edge_index, data.edge_attr

#         x = F.dropout(x, p= dropout_rate, training=self.training)
#         x = self.conv1(x, edge_index, edge_attr)
#         x = F.elu(x)
#         x = F.dropout(x, p= dropout_rate, training=self.training)
#         x = self.conv2(x, edge_index, edge_attr)
#         x = F.elu(x)

#         x = global_mean_pool(x, batch)
#         x = self.lin(x)

#         return F.log_softmax(x, dim=1)


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('Device', device)

GAT model with variable layers of GATCONV (1 or 2)

In [ ]:
class GAT_variable_layer(torch.nn.Module):
    def __init__(self, node_features, hidden_channels, num_heads, num_classes, dropout_rate, use_second_layer):
        super(GAT_variable_layer, self).__init__()
        self.head = num_heads

        self.conv1 = GATConv(node_features, hidden_channels, heads=self.head, dropout=dropout_rate)

        # Conditionally add the second convolutional layer based on the 'use_second_layer' argument
        if use_second_layer:
            self.conv2 = GATConv(hidden_channels * self.head, hidden_channels, heads=self.head,
                                 dropout=dropout_rate)
        else:
            self.conv2 = None

        self.lin = nn.Linear(hidden_channels * self.head, num_classes)

    def forward(self, data, dropout_rate, use_second_layer):
        x, batch, edge_index, edge_attr = data.x, data.batch, data.edge_index, data.edge_attr

        x = F.dropout(x, p=dropout_rate, training=self.training)
        x = self.conv1(x, edge_index, edge_attr)
        x = F.elu(x)
        x = F.dropout(x, p=dropout_rate, training=self.training)

        # Conditionally apply the second convolutional layer
        if use_second_layer and self.conv2 is not None:
            x = self.conv2(x, edge_index, edge_attr)
            x = F.elu(x)

        x = global_mean_pool(x, batch)
        x = self.lin(x)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device', device)

## Training Loop for GAT

Same training loop is for Video, Audio and Early Fusion. Different for Late fusion only

In [ ]:
def train_epoch(train_loader, dr,is_2nd_layer, optimizer, criterion, model_GAT, scheduler):
  loss_train_per = []
  accuracy_train = []
  model_GAT.train()
  for data in train_loader :
        optimizer.zero_grad()
        output_train = model_GAT(data, dropout_rate= dr, use_second_layer=is_2nd_layer)
        target_train = data.y  # Assuming your target labels are stored in 'y'
        loss = criterion(output_train, target_train)
        loss.backward(retain_graph= True)
        optimizer.step()
        predicted_train = output_train.argmax(dim=1)
        accuracy_train.append(accuracy_score(target_train.cpu().numpy(), predicted_train.cpu().numpy()))
        loss_train_per.append(loss.item())
  loss_train_total= np.mean(loss_train_per)
  train_accuracy= np.mean(accuracy_train)
  if scheduler is not None:
        scheduler.step()  # Update learning rate
  return train_accuracy, loss_train_total,model_GAT, optimizer

In [ ]:
def valid_epoch(test_loader, dr, is_2nd_layer, criterion,model_GAT):
  val_loss_per=[]
  accuracy_test=[]
  model_GAT.eval()
  with torch.no_grad():
      for data in test_loader:
          output_test = model_GAT(data, dropout_rate= dr, use_second_layer=is_2nd_layer)
          predicted_test = output_test.argmax(dim=1)
          target_test = data.y
          accuracy_test.append(accuracy_score(target_test.cpu().numpy(), predicted_test.cpu().numpy()))
          loss = criterion(output_test, target_test)
          val_loss_per.append(loss.item())
  val_loss_total= np.mean(val_loss_per)
  val_accuracy= np.mean(accuracy_test)
  return val_accuracy, val_loss_total

In [ ]:
def train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads,is_2nd_layer, trial_subdir, plot_subdir, task_name):
    num_epochs = 20
    best_epoch = -1
    best_stop_epoch=-1
    early_stop_count = 0
    best_val_loss = float('inf')
    best_val_acc = 0
    patience = 3
    batch_size = 1
    all_train_losses = [[] for _ in range(36)]
    all_train_accuracies = [[] for _ in range(36)]
    avg_train_losses=[] # Average of all training losses across all subjects per epoch
    avg_train_accuracies=[] # Average of all training accuracies across all subjects per epoch
    all_val_losses = []
    all_val_accuracies = []
    '''GAT MODEL with 2 layers in architecture
    model_GAT = [GAT(node_features=240, hidden_channels= hidden_classes,num_heads=num_heads, num_classes=3, dropout_rate= dr).to(device) for _ in range(36)]
    '''
    # GAT model with parameters to choose no. of layers in architecture
    model_GAT = [GAT_variable_layer(node_features=240, hidden_channels= hidden_classes,num_heads=num_heads, num_classes=3, dropout_rate= dr, use_second_layer= is_2nd_layer).to(device) for _ in range(36)]
    optimizer = [torch.optim.Adam(model_GAT[_].parameters(), lr= lr) for _ in range(36)]
    scheduler = [StepLR(optimizer[_], step_size=1, gamma=0.9) for _ in range(36)] # decrease LR by a factor of 0.9 every epoch
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        epoch_subdir = os.path.join(trial_subdir, f"epoch_{epoch}")
        os.makedirs(epoch_subdir, exist_ok=True)
        # print(f"## epoch {epoch}  ##")
        val_accuracies =[]
        val_losses=[]
        for idx in range(36):
            # DataLoader setup
            # print(f"## Subject {idx} is test subject  ##")
            train_indices = list(range(36))
            del train_indices[idx]
            train_dataset = [full_dataset[i].to(device) for i in train_indices]
            test_dataset = [full_dataset[idx].to(device)]

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

            # Training loop
            train_accuracy, train_loss, model_GAT[idx], optimizer[idx] = train_epoch(train_loader, dr,is_2nd_layer, optimizer[idx], criterion, model_GAT[idx], scheduler[idx])
            all_train_losses[idx].append(train_loss)
            all_train_accuracies[idx].append(train_accuracy)

            # Validation loop
            val_accuracy,val_loss = valid_epoch(test_loader, dr, is_2nd_layer, criterion, model_GAT[idx])
            val_losses.append(val_loss)
            val_accuracies.append(val_accuracy)
            model_filename = os.path.join(epoch_subdir, f"subject_{idx}_model.pt")
            torch.save({
                        'model_state_dict': model_GAT[idx].state_dict(),
                        'learning rate': lr,
                        'dropout':dr,
                        'hidden_classes':hidden_classes,
                        'num_heads':num_heads,
                        'is_model': is_2nd_layer,
                        }, model_filename)

        all_val_losses.append(np.mean(val_losses))
        all_val_accuracies.append(np.mean(val_accuracies))
        print(f" For epoch {epoch}, Validation Loss: {np.mean(val_losses):.4f}, Validation Accuracy: {np.mean(val_accuracies):.4f}")
        # Early stopping
        if np.mean(val_losses) < best_val_loss:
            best_val_loss = np.mean(val_losses)
            early_stop_count = 0
        else:
            early_stop_count += 1

        if np.mean(val_accuracies) > best_val_acc:
            best_val_acc= np.mean(val_accuracies)
            delete_dir_path= os.path.join(trial_subdir, f"epoch_{best_epoch}")
            if os.path.exists(delete_dir_path):
                  shutil.rmtree(delete_dir_path)
            best_epoch=epoch
        else:
            delete_dir_path= os.path.join(trial_subdir, f"epoch_{epoch}")
            if os.path.exists(delete_dir_path):
                  shutil.rmtree(delete_dir_path)

        if early_stop_count >= patience:
            print(f'Early stopped at epoch: {epoch}')
            best_stop_epoch= epoch
            print(f"Final Validation Accurcay: {best_val_acc}")
            break

    # Calculating Average of all training losses across all subjects per epoch
    for i in range(best_stop_epoch+1):
      avg_train_losses.append(np.average([x[i] for x in all_train_losses]))
      avg_train_accuracies.append(np.average([x[i] for x in all_train_accuracies]))

#     print(f'length of average train losses, {len(avg_train_losses[0])}')
        # Plotting code for all subjects
    fig_loss, axs_loss = plt.subplots(6, 6, figsize=(18, 18))
    fig_acc, axs_acc = plt.subplots(6, 6, figsize=(18, 18))

    for idx in range(36):
        # Plotting Loss
        # epochs = range(len(all_train_losses[idx]))
        ax_loss = axs_loss[idx // 6, idx % 6]
        ax_loss.plot(range(len(all_train_losses[idx])), all_train_losses[idx], label='Train', color='red')
        ax_loss.set_title(f"Loss - Subject {idx}-{task_name}")
        ax_loss.legend()

        # Plotting Accuracy
        # epochs = range(len(all_train_accuracies[idx]))
        ax_acc = axs_acc[idx // 6, idx % 6]
        ax_acc.plot(range(len(all_train_accuracies[idx])), all_train_accuracies[idx], label='Train', color='green')
        ax_acc.set_title(f"Accuracy - Subject {idx}-{task_name}")
        ax_acc.legend()
        # Set integer labels on x-axis
        ax_loss.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
        ax_acc.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))

    # Adjust layout
    fig_loss.tight_layout()
    fig_acc.tight_layout()

    # Validation loss and Validation accuracy curves vs. epoch
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
    ax1.plot(range(len(all_val_losses)), all_val_losses, label='Validation Loss', color='orange')
    ax1.plot(range(len(avg_train_losses)), avg_train_losses, label='Training Loss', color='blue')
    ax1.set_title('Train + Val Loss vs. Epochs')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax2.plot(range(len(all_val_accuracies)), all_val_accuracies, label='Validation Accuracy', color='orange')
    ax2.plot(range(len(avg_train_accuracies)), avg_train_accuracies, label='Training Accuracy', color='blue')
    ax2.set_title('Train + Val Accuracy vs. Epochs')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
    ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
    plt.tight_layout()

    #  Saving the plots
    fig_loss.savefig(os.path.join(plot_subdir+"/"+f"training_loss.png"))
    fig_acc.savefig(os.path.join(plot_subdir+ "/"+ f'training_accuracy.png'))
    fig.savefig(os.path.join(plot_subdir+ "/"+ f'validation_plots.png'))

    # Close the figures
    # plt.close(fig_loss)
    # plt.close(fig_acc)
    # plt.close(fig)

    # Show plots
    plt.show()
    print("Final validation accuracy :", best_val_acc)
    print("Epoch at which Best model with highest validation accuracy is obtained:", best_epoch)
    print("Epochs at which Early stopping is done:", best_stop_epoch)
    return best_val_acc

## Video

Training using Optuna for hyperparameter tuning on video data

In [ ]:
best_trial_dir = None
best_trial_acc = float('-inf')
def objective_ADAM(trial):
    global best_trial_dir, best_trial_acc
    # Hyperparams to tune
    task_name = 'Task5_V'
    lr = trial.suggest_float("lr", 1e-5, 1e-2)
    dr = trial.suggest_float("dropout", 0.1, 0.8)
    hidden_classes = trial.suggest_int('hidden_classes', 16, 180)
    num_heads = trial.suggest_int('num_heads', 1, 8)
    is_2nd_layer = trial.suggest_int("is_2nd_layer", 0,1)
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}, num_heads: {num_heads}, is_2nd_layer: {is_2nd_layer}")
    # print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}, num_heads: {num_heads}")
    save_dir=f'/content/GAT_checkpoints_{task_name}/'
    trial_subdir = os.path.join(save_dir, f"trial_{trial.number}")
    os.makedirs(trial_subdir, exist_ok=True)
    plot_dir=f'/content/GAT_plots_{task_name}/'
    plot_subdir = os.path.join(plot_dir, f"trial_{trial.number}")
    os.makedirs(plot_subdir, exist_ok=True)
    # test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads, trial_subdir)
    test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads,is_2nd_layer, trial_subdir, plot_subdir, task_name)
    if test_acc > best_trial_acc:
      best_trial_acc = test_acc
      remove_trial_subdir= os.path.join(save_dir,f"trial_{best_trial_dir}")
      if os.path.exists(remove_trial_subdir):
          shutil.rmtree(remove_trial_subdir)
      best_trial_dir= trial.number
    else:
        if os.path.exists(trial_subdir):
            shutil.rmtree(trial_subdir)
    return test_acc

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed= seed_value))
study.optimize(objective_ADAM, n_trials= 40 )

print(study.best_trial)

To load model from checkpoints and then directly test it for Video data

In [ ]:
batch_size= 1
task_name = 'Task4_P3_V' # Give the name of task for which you are testing results using checkpoints
save_dir="/kaggle/working/GAT_checkpoints_Task5_A/trial_8/epoch_1" # path to directory where checkpoints are saved
total_correct = 0
total_samples = 0
accuracy_list = []
# Perform 1-subject leave-out cross-validation
for idx in range(36):
    # Load the checkpoint
    checkpoint_path = os.path.join(save_dir, f"subject_{idx}_model.pt")
    checkpoint = torch.load(checkpoint_path)

    # Retrieve parameters
    model_state_dict = checkpoint['model_state_dict']
    lr = checkpoint['learning rate']
    dr = checkpoint['dropout']
    hidden_classes = checkpoint['hidden_classes']
    num_heads = checkpoint['num_heads']
    is_2nd_layer = checkpoint['is_model']
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}, num_heads: {num_heads}, is_2nd_layer: {is_2nd_layer}")

    model_GAT = GAT_variable_layer(node_features=240, hidden_channels=hidden_classes, num_heads=num_heads, num_classes=3, dropout_rate=dr, use_second_layer= is_2nd_layer).to(device)
    # Load the model weights
    model_GAT.load_state_dict(model_state_dict)
    model_GAT.eval()  # Set the model to evaluation mode

    # Create DataLoader for the test dataset
    test_dataset = [full_dataset[idx].to(device)]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Testing loop
    with torch.no_grad():
        for data in test_loader:
            output_test = model_GAT(data, dropout_rate= dr, use_second_layer=is_2nd_layer)
            predicted_test = output_test.argmax(dim=1)
            target_test = data.y
            print("Test Subject:",idx,"Predicted label:", predicted_test, "True label:", target_test)
            total_correct += (predicted_test == target_test).sum().item()
            total_samples += data.y.size(0)
            accuracy_list.append(accuracy_score(target_test.cpu().numpy(), predicted_test.cpu().numpy()))

# Calculate and print test accuracy for the current subject
test_accuracy = total_correct / total_samples
print("total correct :", total_correct," total samples :", total_samples)
print(f"Using Subject {idx} as test, Test Accuracy: {test_accuracy:.4f}")
print("Accuracy using inbuilt function:", np.mean(accuracy_list))

## Audio

Training using Optuna for hyperparameter tuning on Audio data

In [ ]:
best_trial_dir = None
best_trial_acc = float('-inf')
task_name = 'Task5_A'
def objective_ADAM(trial):
    global best_trial_dir, best_trial_acc, task_name
    # Hyperparams to tune
    lr = trial.suggest_float("lr", 1e-5, 1e-2)
    dr = trial.suggest_float("dropout", 0.1, 0.8)
    hidden_classes = trial.suggest_int('hidden_classes', 16, 180)
    num_heads = trial.suggest_int('num_heads', 1, 8)
    is_2nd_layer = trial.suggest_int("is_2nd_layer", 0,1)
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}, num_heads: {num_heads}, is_2nd_layer: {is_2nd_layer}")
    # print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}, num_heads: {num_heads}")
    save_dir=f'/kaggle/working/GAT_checkpoints_{task_name}/'
    trial_subdir = os.path.join(save_dir, f"trial_{trial.number}")
    os.makedirs(trial_subdir, exist_ok=True)
    plot_dir=f'/kaggle/working/GAT_plots_{task_name}/'
    plot_subdir = os.path.join(plot_dir, f"trial_{trial.number}")
    os.makedirs(plot_subdir, exist_ok=True)
    # test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads, trial_subdir)
    test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads,is_2nd_layer, trial_subdir, plot_subdir, task_name)
    if test_acc > best_trial_acc:
      best_trial_acc = test_acc
      remove_trial_subdir= os.path.join(save_dir,f"trial_{best_trial_dir}")
      if os.path.exists(remove_trial_subdir):
          shutil.rmtree(remove_trial_subdir)
      best_trial_dir= trial.number
    else:
        if os.path.exists(trial_subdir):
            shutil.rmtree(trial_subdir)
    return test_acc

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed= seed_value))
study.optimize(objective_ADAM, n_trials= 40 )

print(study.best_trial)

To load model from checkpoints and then directly test it for audio data

In [ ]:
batch_size= 1
task_name = 'Task4_P3_A' # Give the name of task for which you are testing results using checkpoints
save_dir="/kaggle/working/GAT_checkpoints_Task5_A/trial_8/epoch_1" # path to directory where checkpoints are saved
total_correct = 0
total_samples = 0
accuracy_list = []
# Perform 1-subject leave-out cross-validation
for idx in range(36):
    # Load the checkpoint
    checkpoint_path = os.path.join(save_dir, f"subject_{idx}_model.pt")
    checkpoint = torch.load(checkpoint_path)

    # Retrieve parameters
    model_state_dict = checkpoint['model_state_dict']
    lr = checkpoint['learning rate']
    dr = checkpoint['dropout']
    hidden_classes = checkpoint['hidden_classes']
    num_heads = checkpoint['num_heads']
    is_2nd_layer = checkpoint['is_model']
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}, num_heads: {num_heads}, is_2nd_layer: {is_2nd_layer}")

    model_GAT = GAT_variable_layer(node_features=240, hidden_channels=hidden_classes, num_heads=num_heads, num_classes=3, dropout_rate=dr, use_second_layer= is_2nd_layer).to(device)
    # Load the model weights
    model_GAT.load_state_dict(model_state_dict)
    model_GAT.eval()  # Set the model to evaluation mode

    # Create DataLoader for the test dataset
    test_dataset = [full_dataset[idx].to(device)]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Testing loop
    with torch.no_grad():
        for data in test_loader:
            output_test = model_GAT(data, dropout_rate= dr, use_second_layer=is_2nd_layer)
            predicted_test = output_test.argmax(dim=1)
            target_test = data.y
            print("Test Subject:",idx,"Predicted label:", predicted_test, "True label:", target_test)
            total_correct += (predicted_test == target_test).sum().item()
            total_samples += data.y.size(0)
            accuracy_list.append(accuracy_score(target_test.cpu().numpy(), predicted_test.cpu().numpy()))

# Calculate and print test accuracy for the current subject
test_accuracy = total_correct / total_samples
print("total correct :", total_correct," total samples :", total_samples)
print(f"Using Subject {idx} as test, Test Accuracy: {test_accuracy:.4f}")
print("Accuracy using inbuilt function:", np.mean(accuracy_list))

## Early Fusion

Training using Optuna for hyperparameter using on Early fusion of Video and Audio data

In [ ]:
best_trial_dir = None
best_trial_acc = float('-inf')
task_name = 'Task5_E'
def objective_ADAM(trial):
    global best_trial_dir, best_trial_acc, task_name
    # Hyperparams to tune
    lr = trial.suggest_float("lr", 1e-5, 1e-2)
    dr = trial.suggest_float("dropout", 0.1, 0.8)
    hidden_classes = trial.suggest_int('hidden_classes', 16, 180)
    num_heads = trial.suggest_int('num_heads', 1, 8)
    is_2nd_layer = trial.suggest_int("is_2nd_layer", 0,1)
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}, num_heads: {num_heads}, is_2nd_layer: {is_2nd_layer}")
    # print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}, num_heads: {num_heads}")
    save_dir=f'/kaggle/working/GAT_checkpoints_{task_name}/'
    trial_subdir = os.path.join(save_dir, f"trial_{trial.number}")
    os.makedirs(trial_subdir, exist_ok=True)
    plot_dir=f'/kaggle/working/GAT_plots_{task_name}/'
    plot_subdir = os.path.join(plot_dir, f"trial_{trial.number}")
    os.makedirs(plot_subdir, exist_ok=True)
    # test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads, trial_subdir)
    test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads,is_2nd_layer, trial_subdir, plot_subdir, task_name)
    if test_acc > best_trial_acc:
      best_trial_acc = test_acc
      remove_trial_subdir= os.path.join(save_dir,f"trial_{best_trial_dir}")
      if os.path.exists(remove_trial_subdir):
          shutil.rmtree(remove_trial_subdir)
      best_trial_dir= trial.number
    else:
        if os.path.exists(trial_subdir):
            shutil.rmtree(trial_subdir)
    return test_acc

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed= seed_value))
study.optimize(objective_ADAM, n_trials= 40 )

print(study.best_trial)

To load model from checkpoints and then directly test it for Early fusion of Video and Audio data

In [ ]:
batch_size= 1
task_name = 'Task4_P3_E' # Give the name of task for which you are testing results using checkpoints
save_dir="/kaggle/working/GAT_checkpoints_Task5_A/trial_8/epoch_1" # path to directory where checkpoints are saved
total_correct = 0
total_samples = 0
accuracy_list = []
# Perform 1-subject leave-out cross-validation
for idx in range(36):
    # Load the checkpoint
    checkpoint_path = os.path.join(save_dir, f"subject_{idx}_model.pt")
    checkpoint = torch.load(checkpoint_path)

    # Retrieve parameters
    model_state_dict = checkpoint['model_state_dict']
    lr = checkpoint['learning rate']
    dr = checkpoint['dropout']
    hidden_classes = checkpoint['hidden_classes']
    num_heads = checkpoint['num_heads']
    is_2nd_layer = checkpoint['is_model']
    print(f"learning rate:{lr},dropout: {dr}, hidden_classes: {hidden_classes}, num_heads: {num_heads}, is_2nd_layer: {is_2nd_layer}")

    model_GAT = GAT_variable_layer(node_features=240, hidden_channels=hidden_classes, num_heads=num_heads, num_classes=3, dropout_rate=dr, use_second_layer= is_2nd_layer).to(device)
    # Load the model weights
    model_GAT.load_state_dict(model_state_dict)
    model_GAT.eval()  # Set the model to evaluation mode

    # Create DataLoader for the test dataset
    test_dataset = [full_dataset[idx].to(device)]
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Testing loop
    with torch.no_grad():
        for data in test_loader:
            output_test = model_GAT(data, dropout_rate= dr, use_second_layer=is_2nd_layer)
            predicted_test = output_test.argmax(dim=1)
            target_test = data.y
            print("Test Subject:",idx,"Predicted label:", predicted_test, "True label:", target_test)
            total_correct += (predicted_test == target_test).sum().item()
            total_samples += data.y.size(0)
            accuracy_list.append(accuracy_score(target_test.cpu().numpy(), predicted_test.cpu().numpy()))

# Calculate and print test accuracy for the current subject
test_accuracy = total_correct / total_samples
print("total correct :", total_correct," total samples :", total_samples)
print(f"Using Subject {idx} as test, Test Accuracy: {test_accuracy:.4f}")
print("Accuracy using inbuilt function:", np.mean(accuracy_list))

## Late Fusion

For late fusion the training and validation loop are different from previous training and validation loops.

In [ ]:
def train_epoch_late(train_loader1, dr1,is_2nd_layer1, optimizer1, model_GAT1, scheduler1, train_loader2, dr2,is_2nd_layer2, optimizer2, model_GAT2, scheduler2,criterion):
    loss_train_per = []
    accuracy_train = []
    model_GAT1.train()
    model_GAT2.train()
    for data1, data2 in zip(train_loader1, train_loader2):
        optimizer1.zero_grad()
        output_train1 = model_GAT1(data1, dropout_rate= dr1, use_second_layer=is_2nd_layer1)
        target_train1 = data1.y
        loss1 = criterion(output_train1, target_train1)
        loss1.backward(retain_graph= True)
        optimizer1.step()

        optimizer2.zero_grad()
        output_train2 = model_GAT2(data2, dropout_rate= dr2, use_second_layer=is_2nd_layer2)
        target_train2 = data2.y
        loss2 = criterion(output_train2, target_train2)
        loss2.backward(retain_graph= True)
        optimizer2.step()
        final_output = (output_train1+output_train2)/2.0
        predicted_labels = torch.argmax(final_output, dim=1)
        target_fused = torch.max(target_train1, target_train2)
        loss = (loss1.item()+ loss2.item())/2.0
        loss_train_per.append(loss)
        accuracy_train.append(accuracy_score(target_fused.cpu().numpy(), predicted_labels.cpu().numpy()))
    loss_train_total= np.mean(loss_train_per)
    train_accuracy= np.mean(accuracy_train)
    if scheduler1 is not None:
        scheduler1.step()  # Update learning rate
    if scheduler2 is not None:
        scheduler2.step()  # Update learning rate

    return train_accuracy, loss_train_total,model_GAT1, optimizer1, model_GAT2, optimizer2

In [ ]:
def valid_epoch_late(test_loader1, dr1, is_2nd_layer1, model_GAT1, test_loader2, dr2, is_2nd_layer2, model_GAT2, criterion):
  val_loss_per=[]
  accuracy_test=[]
  model_GAT1.eval()
  model_GAT2.eval()
  with torch.no_grad():
      for data1, data2 in zip(test_loader1, test_loader2):
        output_test1 = model_GAT1(data1, dropout_rate= dr2, use_second_layer=is_2nd_layer2)
        target_test1 = data1.y
        output_test2 = model_GAT2(data2, dropout_rate= dr2, use_second_layer=is_2nd_layer2)
        target_test2 = data2.y
        output_test_fused = (output_test1+output_test2)/2.0
        predicted_labels = torch.argmax(output_test_fused, dim=1)
        target_fused = torch.max(target_test1, target_test2)
        accuracy_test.append(accuracy_score(target_fused.cpu().numpy(), predicted_labels.cpu().numpy()))
        loss = criterion(output_test_fused, target_fused)
        val_loss_per.append(loss.item())
  val_loss_total= np.mean(val_loss_per)
  val_accuracy= np.mean(accuracy_test)
  return val_accuracy, val_loss_total

In [ ]:
def train_valid_loop_ADAM_late(lr1, dr1, hidden_classes1,num_heads1,is_2nd_layer1, lr2, dr2, hidden_classes2,num_heads2,is_2nd_layer2, trial_subdir, plot_subdir, task_name):
    num_epochs = 20
    best_epoch = -1
    best_stop_epoch=-1
    early_stop_count = 0
    best_val_loss = float('inf')
    best_val_acc = 0
    patience = 3
    batch_size = 1
    all_train_losses = [[] for _ in range(36)]
    all_train_accuracies = [[] for _ in range(36)]
    avg_train_losses=[] # Average of all training losses across all subjects per epoch
    avg_train_accuracies=[] # Average of all training accuracies across all subjects per epoch
    all_val_losses = []
    all_val_accuracies = []
    '''GAT MODEL with 2 layers in architecture
    model_GAT = [GAT(node_features=240, hidden_channels= hidden_classes,num_heads=num_heads, num_classes=3, dropout_rate= dr).to(device) for _ in range(36)]
    '''
    # GAT model with parameters to choose no. of layers in architecture
    model_GAT1 = [GAT_variable_layer(node_features=240, hidden_channels= hidden_classes1,num_heads=num_heads1, num_classes=3, dropout_rate= dr1, use_second_layer= is_2nd_layer1).to(device) for _ in range(36)]
    optimizer1 = [torch.optim.Adam(model_GAT1[_].parameters(), lr= lr1) for _ in range(36)]
    scheduler1 = [StepLR(optimizer1[_], step_size=1, gamma=0.9) for _ in range(36)] # decrease LR by a factor of 0.9 every epoch

    model_GAT2 = [GAT_variable_layer(node_features=240, hidden_channels= hidden_classes2,num_heads=num_heads2, num_classes=3, dropout_rate= dr2, use_second_layer= is_2nd_layer2).to(device) for _ in range(36)]
    optimizer2 = [torch.optim.Adam(model_GAT2[_].parameters(), lr= lr2) for _ in range(36)]
    scheduler2 = [StepLR(optimizer2[_], step_size=1, gamma=0.9) for _ in range(36)] # decrease LR by a factor of 0.9 every epoch
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        epoch_subdir = os.path.join(trial_subdir, f"epoch_{epoch}")
        os.makedirs(epoch_subdir, exist_ok=True)
        # print(f"## epoch {epoch}  ##")
        val_accuracies =[]
        val_losses=[]
        for idx in range(36):
            # DataLoader setup
            # print(f"## Subject {idx} is test subject  ##")
            train_indices = list(range(36))
            del train_indices[idx]
            train_dataset1 = [full_dataset_V[i].to(device) for i in train_indices]
            test_dataset1 = [full_dataset_V[idx].to(device)]
            train_dataset2 = [full_dataset_A[i].to(device) for i in train_indices]
            test_dataset2 = [full_dataset_A[idx].to(device)]

            train_loader1 = DataLoader(train_dataset1, batch_size=batch_size, shuffle=True)
            test_loader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=False)
            train_loader2 = DataLoader(train_dataset2, batch_size=batch_size, shuffle=True)
            test_loader2 = DataLoader(test_dataset2, batch_size=batch_size, shuffle=False)

            # Training loop
            train_accuracy, train_loss, model_GAT1[idx], optimizer1[idx], model_GAT2[idx], optimizer2[idx] = train_epoch_late(train_loader1, dr1,is_2nd_layer1, optimizer1[idx], model_GAT1[idx], scheduler1[idx], train_loader2, dr2,is_2nd_layer2, optimizer2[idx], model_GAT2[idx], scheduler2[idx],criterion)
            all_train_losses[idx].append(train_loss)
            all_train_accuracies[idx].append(train_accuracy)

            # Validation loop
            val_accuracy,val_loss = valid_epoch_late(test_loader1, dr1, is_2nd_layer1, model_GAT1[idx], test_loader2, dr2, is_2nd_layer2, model_GAT2[idx], criterion)
            val_losses.append(val_loss)
            val_accuracies.append(val_accuracy)
            # Saving model checkpoints
            model_filename1 = os.path.join(epoch_subdir, f"Model1_subject_{idx}_model.pt")
            torch.save({
                        'model_state_dict1': model_GAT1[idx].state_dict(),
                        'learning rate1': lr1,
                        'dropout1':dr1,
                        'hidden_classes1':hidden_classes1,
                        'num_heads1':num_heads1,
                        'is_model1': is_2nd_layer1,
                        }, model_filename1)
            model_filename2 = os.path.join(epoch_subdir, f"Model2_subject_{idx}_model.pt")
            torch.save({
                        'model_state_dict2': model_GAT2[idx].state_dict(),
                        'learning rate2': lr2,
                        'dropout2':dr2,
                        'hidden_classes2':hidden_classes2,
                        'num_heads2':num_heads2,
                        'is_model2': is_2nd_layer2,
                        }, model_filename2)

        all_val_losses.append(np.mean(val_losses))
        all_val_accuracies.append(np.mean(val_accuracies))
        print(f" For epoch {epoch}, Validation Loss: {np.mean(val_losses):.4f}, Validation Accuracy: {np.mean(val_accuracies):.4f}")

        # Early stopping
        if np.mean(val_losses) < best_val_loss:
            best_val_loss = np.mean(val_losses)
            early_stop_count = 0
        else:
            early_stop_count += 1

        if np.mean(val_accuracies) > best_val_acc:
            best_val_acc= np.mean(val_accuracies)
            delete_dir_path= os.path.join(trial_subdir, f"epoch_{best_epoch}")
            if os.path.exists(delete_dir_path):
                  shutil.rmtree(delete_dir_path)
            best_epoch=epoch
        else:
            delete_dir_path= os.path.join(trial_subdir, f"epoch_{epoch}")
            if os.path.exists(delete_dir_path):
                  shutil.rmtree(delete_dir_path)

        if early_stop_count >= patience:
            print(f'Early stopped at epoch: {epoch}')
            best_stop_epoch= epoch
            print(f"Final Validation Accurcay: {best_val_acc}")
            break

    # Calculating Average of all training losses across all subjects per epoch
    for i in range(best_stop_epoch+1):
      avg_train_losses.append(np.average([x[i] for x in all_train_losses]))
      avg_train_accuracies.append(np.average([x[i] for x in all_train_accuracies]))

#     print(f'length of average train losses, {len(avg_train_losses[0])}')
        # Plotting code for all subjects
    fig_loss, axs_loss = plt.subplots(6, 6, figsize=(18, 18))
    fig_acc, axs_acc = plt.subplots(6, 6, figsize=(18, 18))

    for idx in range(36):
        # Plotting Loss
        # epochs = range(len(all_train_losses[idx]))
        ax_loss = axs_loss[idx // 6, idx % 6]
        ax_loss.plot(range(len(all_train_losses[idx])), all_train_losses[idx], label='Train', color='red')
        ax_loss.set_title(f"Loss - Subject {idx}-{task_name}")
        ax_loss.legend()

        # Plotting Accuracy
        # epochs = range(len(all_train_accuracies[idx]))
        ax_acc = axs_acc[idx // 6, idx % 6]
        ax_acc.plot(range(len(all_train_accuracies[idx])), all_train_accuracies[idx], label='Train', color='green')
        ax_acc.set_title(f"Accuracy - Subject {idx}-{task_name}")
        ax_acc.legend()
        # Set integer labels on x-axis
        ax_loss.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
        ax_acc.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))

    # Adjust layout
    fig_loss.tight_layout()
    fig_acc.tight_layout()

    # Validation loss and Validation accuracy curves vs. epoch
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))
    ax1.plot(range(len(all_val_losses)), all_val_losses, label='Validation Loss', color='orange')
    ax1.plot(range(len(avg_train_losses)), avg_train_losses, label='Training Loss', color='blue')
    ax1.set_title('Train + Val Loss vs. Epochs')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax2.plot(range(len(all_val_accuracies)), all_val_accuracies, label='Validation Accuracy', color='orange')
    ax2.plot(range(len(avg_train_accuracies)), avg_train_accuracies, label='Training Accuracy', color='blue')
    ax2.set_title('Train + Val Accuracy vs. Epochs')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
    ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: int(x)))
    plt.tight_layout()

    #  Saving the plots
    fig_loss.savefig(os.path.join(plot_subdir+"/"+f"training_loss.png"))
    fig_acc.savefig(os.path.join(plot_subdir+ "/"+ f'training_accuracy.png'))
    fig.savefig(os.path.join(plot_subdir+ "/"+ f'validation_plots.png'))

    # Show plots
    plt.show()
    print("Final validation accuracy :", best_val_acc)
    print("Epoch at which Best model with highest validation accuracy is obtained:", best_epoch)
    print("Epochs at which Early stopping is done:", best_stop_epoch)
    return best_val_acc

Training using Optuna for hyperparameter tuning on Late fusion of video and Audio data

In [ ]:
best_trial_dir = None
best_trial_acc = float('-inf')
task_name = 'Task5_L'
def objective_ADAM_late(trial):
    global best_trial_dir, best_trial_acc, task_name
    # Hyperparams to tune
    lr1 = trial.suggest_float("lr1", 1e-5, 1e-2)
    lr2 = trial.suggest_float("lr2", 1e-5, 1e-2)
    dr1 = trial.suggest_float("dropout1", 0.1, 0.8)
    dr2 = trial.suggest_float("dropout2", 0.1, 0.8)
    hidden_classes1 = trial.suggest_int("hidden_classes1", 16, 180)
    hidden_classes2 = trial.suggest_int("hidden_classes2", 16, 180)
    num_heads1= trial.suggest_int("num_heads1", 1,8)
    num_heads2= trial.suggest_int("num_heads2", 1,8)
    is_2nd_layer1 = trial.suggest_int("is_2nd_layer1", 0,1)
    is_2nd_layer2 = trial.suggest_int("is_2nd_layer2", 0,1)
    print("learning rate 1:", lr1,"learning rate 2:", lr2,"dropout rate1:", dr1, "dropout rate2:", dr2,"hidden_classes1 :", hidden_classes1, "hidden_classes2:", hidden_classes2, "num_heads1:",num_heads1, "num_heads2:",num_heads2, "is_2nd_layer1: ",is_2nd_layer1, "is_2nd_layer2: ",is_2nd_layer2)
    save_dir=f'/kaggle/working/GAT_checkpoints_{task_name}/'
    trial_subdir = os.path.join(save_dir, f"trial_{trial.number}")
    os.makedirs(trial_subdir, exist_ok=True)
    plot_dir=f'/kaggle/working/GAT_plots_{task_name}/'
    plot_subdir = os.path.join(plot_dir, f"trial_{trial.number}")
    os.makedirs(plot_subdir, exist_ok=True)
    # test_acc = train_valid_loop_ADAM(lr, dr, hidden_classes,num_heads, trial_subdir)
    test_acc = train_valid_loop_ADAM_late(lr1, dr1, hidden_classes1,num_heads1,is_2nd_layer1, lr2, dr2, hidden_classes2,num_heads2,is_2nd_layer2, trial_subdir, plot_subdir, task_name)
    if test_acc > best_trial_acc:
      best_trial_acc = test_acc
      remove_trial_subdir= os.path.join(save_dir,f"trial_{best_trial_dir}")
      if os.path.exists(remove_trial_subdir):
          shutil.rmtree(remove_trial_subdir)
      best_trial_dir= trial.number
    else:
        if os.path.exists(trial_subdir):
            shutil.rmtree(trial_subdir)
    return test_acc

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed= seed_value))
study.optimize(objective_ADAM_late, n_trials= 40 )

print(study.best_trial)

To load model from checkpoints and then directly test it for late fusion of video and Audio data

In [ ]:
batch_size=1
task_name = 'Task5_L' # name of task for which you are testing
save_dir="/content/drive/MyDrive/GAT_checkpoints/Late Fusion/trial_23" # path of directory where checkpoints are saved
print(f" dropout1: {dr1}, dropout2: {dr2}, hidden_classes1: {hidden_classes1}, hidden_classes2: {hidden_classes2}, num_heads1: {num_heads1}, num_heads2: {num_heads2}")
model_GAT1 = GAT(node_features=240, hidden_channels= hidden_classes1, num_heads=num_heads1, num_classes=3, dropout_rate=dr1).to(device)
model_GAT2 = GAT(node_features=240, hidden_channels= hidden_classes2, num_heads=num_heads2, num_classes=3, dropout_rate=dr2).to(device)
total_correct = 0
total_samples = 0
test_accuracy = []
# Perform 1-subject leave-out cross-validation
for idx in range(36):
    # Load the checkpoint
    checkpoint_path1 = os.path.join(save_dir, f"Model1_subject_{idx}_model.pt")
    checkpoint1 = torch.load(checkpoint_path1)
    # Retrieve parameters
    model_state_dict1 = checkpoint1['model_state_dict']
    lr1 = checkpoint1['learning rate']
    dr1 = checkpoint1['dropout']
    hidden_classes1 = checkpoint1['hidden_classes']
    num_heads1 = checkpoint1['num_heads']
    is_2nd_layer1 = checkpoint1['is_model']
    print(f"learning rate:{lr1},dropout: {dr1}, hidden_classes: {hidden_classes1}, num_heads: {num_heads1}, is_2nd_layer: {is_2nd_layer1}")

    model_GAT1 = GAT_variable_layer(node_features=240, hidden_channels=hidden_classes1, num_heads=num_heads1, num_classes=3, dropout_rate=dr1, use_second_layer= is_2nd_layer1).to(device)
    # Load the model weights
    model_GAT1.load_state_dict(model_state_dict1)
    model_GAT1.eval()  # Set the model to evaluation mode


    checkpoint_path2 = os.path.join(save_dir, f"Model2_subject_{idx}_model.pt")
    checkpoint2 = torch.load(checkpoint_path2)
    # Retrieve parameters
    model_state_dict2 = checkpoint2['model_state_dict']
    lr2 = checkpoint2['learning rate']
    dr2 = checkpoint2['dropout']
    hidden_classes2 = checkpoint2['hidden_classes']
    num_heads2 = checkpoint2['num_heads']
    is_2nd_layer2 = checkpoint2['is_model']
    print(f"learning rate:{lr2},dropout: {dr2}, hidden_classes: {hidden_classes2}, num_heads: {num_heads2}, is_2nd_layer: {is_2nd_layer2}")

    model_GAT2 = GAT_variable_layer(node_features=240, hidden_channels=hidden_classes2, num_heads=num_heads2, num_classes=3, dropout_rate=dr2, use_second_layer= is_2nd_layer2).to(device)
    # Load the model weights
    model_GAT2.load_state_dict(model_state_dict2)
    model_GAT2.eval()  # Set the model to evaluation mode

    # Create DataLoader for the test dataset
    test_dataset1 = [full_dataset_V[idx].to(device)]
    test_loader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=False)
    test_dataset2 = [full_dataset_A[idx].to(device)]
    test_loader2 = DataLoader(test_dataset2, batch_size=batch_size, shuffle=False)

    # Testing loop
    model_GAT1.eval()
    model_GAT2.eval()
    with torch.no_grad():
        for data1, data2 in zip(test_loader1, test_loader2):
            output_test1 = model_GAT1(data1, dropout_rate= dr1)
            target_test1 = data1.y
            output_test2 = model_GAT2(data2, dropout_rate= dr2)
            target_test2 = data2.y
            final_output = (output_test1+output_test2)/2.0
            predicted_labels = torch.argmax(final_output, dim=1)
            target_fused = torch.max(target_test1, target_test2)
            total_correct += (predicted_labels == target_fused).sum().item()
            total_samples += data1.y.size(0)
            print("Test Subject:",idx," predicted label:", predicted_labels ," True label : ", target_fused)
            test_accuracy.append(accuracy_score(target_fused.cpu().numpy(), predicted_labels.cpu().numpy()))

# Calculate and print test accuracy for the current subject
test_accuracy = total_correct / total_samples
print("total correct :", total_correct," total samples :", total_samples)
print(f"Using Subject {idx} as test, Test Accuracy: {test_accuracy:.4f}")
print("Accuracy using inbuilt function:", np.mean(test_accuracy))

# ML Models

In [ ]:
# all imports go here
import pandas as pd
import os
import pickle
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
import numpy as np
import xgboost as xgb
!pip install boruta
from boruta import BorutaPy
from xgboost import plot_importance
import matplotlib.pyplot as plt
from sklearn import tree
from scipy.stats import pearsonr
from scipy.signal import savgol_filter

In [ ]:
!pip install optuna
import optuna
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import LeaveOneOut, LeaveOneGroupOut
from sklearn.impute import SimpleImputer
# prompt: ignore warnings

import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)
RANDOM_STATE = 42

In [ ]:
'''
Defining Labels from values as per distribution
values   labels
0-2         0
3-4         1
5-19        2
'''
labels=[]
values=[13,4,19,14,3,3,2,4,7,1,5,4,0,0,3,4,2,13,8,0,2,2,1,6,3,2,3,4,2,4,2,0,7,1,13,2]
for i in values:
  if (i<=2):
        labels.append(0)
  elif (i>=5):
        labels.append(2)
  else :
        labels.append(1)
print(labels)
print(len(set(labels)))

In [ ]:
# Specify the directory containing CSV files
directory_path = '/content/drive/MyDrive/Cambridge_Data/Modify_Audio_Data/Task5p_A'
files=os.listdir(directory_path)
files.sort()
media = 'audio'
# Define a function to calculate auto-correlation
def auto_correlation(series, lag=1):
    return np.corrcoef(series[:-lag], series[lag:])[0, 1]

# Initialize an empty DataFrame to store results for each subject
subject_results = pd.DataFrame()

# Loop through each CSV file in the directory
for file_name in files:
      file_path = os.path.join(directory_path, file_name)

      # Load CSV data into a pandas DataFrame
      df = pd.read_csv(file_path)
      if media=='audio':
          df.columns=['Unnamed: 0','MFCC 1', 'MFCC 2', 'MFCC 3', 'MFCC 4', 'MFCC 5', 'MFCC 6', 'MFCC 7', 'MFCC 8', 'MFCC 9', 'MFCC 10', 'MFCC 11', 'MFCC 12', 'MFCC 13', 'MFCC Delta 1', 'MFCC Delta 2', 'MFCC Delta 3', 'MFCC Delta 4', 'MFCC Delta 5', 'MFCC Delta 6', 'MFCC Delta 7', 'MFCC Delta 8', 'MFCC Delta 9', 'MFCC Delta 10', 'MFCC Delta 11', 'MFCC Delta 12', 'MFCC Delta 13', 'MFCC Delta Delta 1', 'MFCC Delta Delta 2', 'MFCC Delta Delta 3', 'MFCC Delta Delta 4', 'MFCC Delta Delta 5', 'MFCC Delta Delta 6', 'MFCC Delta Delta 7', 'MFCC Delta Delta 8', 'MFCC Delta Delta 9', 'MFCC Delta Delta 10', 'MFCC Delta Delta 11', 'MFCC Delta Delta 12', 'MFCC Delta Delta 13', 'GTCC 1', 'GTCC 2', 'GTCC 3', 'GTCC 4', 'GTCC 5', 'GTCC 6', 'GTCC 7', 'GTCC 8', 'GTCC 9', 'GTCC 10', 'GTCC 11', 'GTCC 12', 'GTCC 13', 'GTCC Delta 1', 'GTCC Delta 2', 'GTCC Delta 3', 'GTCC Delta 4', 'GTCC Delta 5', 'GTCC Delta 6', 'GTCC Delta 7', 'GTCC Delta 8', 'GTCC Delta 9', 'GTCC Delta 10', 'GTCC Delta 11', 'GTCC Delta 12', 'GTCC Delta 13', 'GTCC Delta Delta 1','GTCC Delta Delta 2','GTCC Delta Delta 3', 'GTCC Delta Delta 4', 'GTCC Delta Delta 5', 'GTCC Delta Delta 6', 'GTCC Delta Delta 7', 'GTCC Delta Delta 8', 'GTCC Delta Delta 9', 'GTCC Delta Delta 10', 'GTCC Delta Delta 11', 'GTCC Delta Delta 12', 'GTCC Delta Delta 13', 'Spectral Centroid', 'Spectral Crest', 'Spectral Decrease', 'Spectral Entropy', 'Spectral Flatness', 'Spectral Flux', 'Spectral Kurtosis', 'Spectral Rolloff', 'Spectral Skewness', 'Spectral Slope', 'Spectral Spread', 'Pitch', 'Harmonic Ratio']
      df.drop(['Unnamed: 0'], axis=1, inplace=True)
      # # Create a dictionary to store statistical features for the current subject
      # subject_features = {'Subject': file_name.split('.')[0]}
      subject_features = pd.DataFrame(columns=['Subject'])
      subject_features['Subject'] = [file_name.split('.')[0]]

      # Calculate statistics for each feature and its derivatives
      for column in df.columns:
          feature_data = df[column]

          # Original Feature
          subject_features[f'Avg_{column}'] = np.mean(feature_data)
          subject_features[f'Max_{column}'] = np.max(feature_data)
          subject_features[f'Min_{column}'] = np.min(feature_data)
          subject_features[f'Auto-Correlation_{column}'] = auto_correlation(feature_data)
          subject_features[f'Std_Dev_{column}'] = np.std(feature_data)
          subject_features[f'Median_{column}'] = np.median(feature_data)

          # First Derivative
          first_derivative = np.gradient(feature_data)
          subject_features[f'Avg_Derivative1_{column}'] = np.mean(first_derivative)
          subject_features[f'Max_Derivative1_{column}'] = np.max(first_derivative)
          subject_features[f'Min_Derivative1_{column}'] = np.min(first_derivative)
          subject_features[f'Auto-Correlation_Derivative1_{column}'] = auto_correlation(first_derivative)
          subject_features[f'Std_Dev_Derivative1_{column}'] = np.std(first_derivative)
          subject_features[f'Median_Derivative1_{column}'] = np.median(first_derivative)

          # Second Derivative
          second_derivative = np.gradient(first_derivative)
          subject_features[f'Avg_Derivative2_{column}'] = np.mean(second_derivative)
          subject_features[f'Max_Derivative2_{column}'] = np.max(second_derivative)
          subject_features[f'Min_Derivative2_{column}'] = np.min(second_derivative)
          subject_features[f'Auto-Correlation_Derivative2_{column}'] = auto_correlation(second_derivative)
          subject_features[f'Std_Dev_Derivative2_{column}'] = np.std(second_derivative)
          subject_features[f'Median_Derivative2_{column}'] = np.median(second_derivative)

      # Append the features for the current subject to the DataFrame
      subject_results = subject_results._append(subject_features, ignore_index=True)

# Display or further process subject_results DataFrame as needed
print(subject_results)

In [ ]:
# Drop subject column from a dataframe as it is not a feature
subject_results.drop('Subject', axis=1, inplace=True)

In [ ]:
#  Replace Nan values with mean values of a column
# Create an imputer transformer
imputer = SimpleImputer(strategy='mean')

# Fit the imputer to the data
imputer.fit(subject_results)

# Transform the data
# subject_results = imputer.transform(subject_results)
subject_results = pd.DataFrame(imputer.transform(subject_results), columns=subject_results.columns, index=subject_results.index)

In [ ]:
for train_index, test_index in LeaveOneOut().split(subject_results, labels):
#       X_train_features = subject_results.iloc[train_index].columns
#       X_test_features = subject_results.iloc[test_index].columns
      X_train, X_test = subject_results.iloc[train_index], subject_results.iloc[test_index]
      y_train, y_test = np.array(labels)[train_index], np.array(labels)[test_index]

In [ ]:
# Function for testing of ML models
def test_results(clf_model, best_params,subject_results, labels):
      if clf_model == 'logistic_regression':
        optimal_clf = LogisticRegression(**best_params)

      elif clf_model == 'linear_svm':
        optimal_clf = SVC(kernel='linear', probability=True, **best_params)

      elif clf_model == 'rbf_svm':
        optimal_clf = SVC(kernel='rbf', probability=True, **best_params)

      elif clf_model == 'decision_tree':
        optimal_clf = DecisionTreeClassifier(**best_params)

      elif clf_model == 'rforest':
        optimal_clf = RandomForestClassifier(**best_params)

      elif clf_model == 'adaboost':
        optimal_clf = AdaBoostClassifier(**best_params)

      elif clf_model == 'xgboost':
        optimal_clf = xgb.XGBClassifier(**best_params)

      elif clf_model == 'bagging':
        optimal_clf = BaggingClassifier(**best_params)

      optimal_clf.fit(subject_results, labels)
      y_pred = optimal_clf.predict(subject_results)
      print("Model:",clf_model, 'Test Accuracy:', accuracy_score(labels, y_pred))

In [ ]:
#  Optuna for hyperparameter tuning of ML models
def objective(trial, clf_name, subject_results, labels):
    if clf_name == 'logistic_regression':
        C = trial.suggest_loguniform('C', 1e-2, 1e+2)
        solver= trial.suggest_categorical("solver",['liblinear','newton-cg','lbfgs','sag','saga'])
        clf_model = LogisticRegression(C=C, max_iter=10000, solver=solver , random_state=RANDOM_STATE)

    elif clf_name == 'linear_svm':
        C = trial.suggest_loguniform('C', 1e-2, 1e+2)
        degree = trial.suggest_int('degree',1, 50)
        gamma = trial.suggest_loguniform('gamma',0.001,10000)
        clf_model = SVC(C=C, kernel='linear', degree=degree,gamma=gamma, random_state=RANDOM_STATE)

    elif clf_name == 'rbf_svm':
        C = trial.suggest_loguniform('C', 1e-2, 1e+2)
        degree = trial.suggest_int('degree',1, 50)
        gamma = trial.suggest_loguniform('gamma',1e-2,1e+2)
        clf_model = SVC(C=C, kernel='rbf', degree=degree,gamma=gamma, random_state=RANDOM_STATE)

    elif clf_name == 'decision_tree':
        max_depth = trial.suggest_int('max_depth', 2, 20)
        clf_model = DecisionTreeClassifier(max_depth=max_depth, random_state=RANDOM_STATE)

    elif clf_name == 'rforest':
        n_estimators = trial.suggest_int('n_estimators', 2, 20)
        max_depth = int(trial.suggest_int('max_depth', 1, 32, log=True))
        clf_model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=RANDOM_STATE)

    elif clf_name == 'adaboost':
        n_estimators = trial.suggest_int('n_estimators', 20, 100)
        clf_model = AdaBoostClassifier(n_estimators=n_estimators, random_state=RANDOM_STATE)

    elif clf_name == 'xgboost':
        n_estimators = trial.suggest_int('n_estimators', 20, 100)
        clf_model = xgb.XGBClassifier(n_estimators=n_estimators, random_state=RANDOM_STATE)

    elif clf_name == 'bagging':
        n_estimators = trial.suggest_int('n_estimators', 20, 100)
        clf_model = BaggingClassifier(n_estimators=n_estimators, random_state=RANDOM_STATE)

    scores = []
    for train_index, test_index in LeaveOneOut().split(subject_results, labels):
#       X_train_features = subject_results.iloc[train_index].columns
#       X_test_features = subject_results.iloc[test_index].columns
      X_train, X_test = subject_results.iloc[train_index], subject_results.iloc[test_index]
      y_train, y_test = np.array(labels)[train_index], np.array(labels)[test_index]
      # get the scaled version of the train and test set
#       mm = MinMaxScaler()
#       X_train_scaled = pd.DataFrame(mm.fit_transform(X_train), columns= X_train_features)
#       X_test_scaled = pd.DataFrame(mm.transform(X_test), columns=X_test_features)
      clf_model.fit(X_train, y_train)
      score = accuracy_score(y_test, clf_model.predict(X_test))
      scores.append(score)
    return np.mean(scores)


# define models
MODELS = ['logistic_regression',
         'linear_svm',
         'rbf_svm',
         'decision_tree',
          'rforest',
         'adaboost',
         'xgboost',
         'bagging']

for clf_model in MODELS:
    # print clf_model
    print(clf_model)
    sampler = optuna.samplers.TPESampler(multivariate=True)
    # default is Tree-structured Parzen Estimator (TPE) optimization algorithm
    study = optuna.create_study(direction='maximize',
                              sampler=sampler,
                              study_name=clf_model,
                              pruner=optuna.pruners.HyperbandPruner(min_resource=1, reduction_factor=3))
    ## we use X_train in the following, not X_train_scaled, because optuna pipeline in objective function has MinMaxScaler() already
    study.optimize(lambda trial:objective(trial, clf_model,subject_results, labels), n_trials= 35)
    trial = study.best_trial
    best_params = trial.params
    print("Model:",clf_model,"Best accuracy:", study.best_value,"Best parameters:", study.best_params)
    test_results(clf_model, best_params, subject_results, labels)